In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostClassifier, cv

In [2]:
# !pip install scikit-optimize

In [3]:
train_1mm = pd.read_csv('ProjectTrainingData-1MM.csv', header=None)
val_50k = pd.read_csv('ProjectValidationData.csv', header = None)
# train_1mm = pd.read_csv('cleanedTraining.csv', header=0)
# train_1mm = pd.read_csv('ProjectTrainingData.csv', header=None)

In [4]:
train_1mm.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,4803607860744373534,0,14102809,1005,0,85f751fd,c4e18dd6,50e219e0,f0d41ff1,2347f47a,...,1,2,23143,320,50,2665,0,34,100034,221
1,13386437727020255392,0,14102705,1005,1,0a742914,510bd839,f028772b,ecad2386,7801e8d9,...,1,0,21412,320,50,2467,2,167,100077,23
2,18034954954610732886,1,14102817,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,22814,320,50,2647,2,35,100148,23
3,6676508794134781022,0,14102816,1005,0,85f751fd,c4e18dd6,50e219e0,39947756,2347f47a,...,1,3,21801,320,50,2515,1,161,100191,71
4,5503744330236741468,1,14102107,1005,1,5b4d2eda,16a36ef3,f028772b,ecad2386,7801e8d9,...,1,0,16208,320,50,1800,3,167,100077,23


In [5]:
train_1mm.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 

           'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15',  

           'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

val_50k.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 

           'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15',  

           'C16', 'C17', 'C18', 'C19', 'C20', 'C21']


train_1mm.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,4803607860744373534,0,14102809,1005,0,85f751fd,c4e18dd6,50e219e0,f0d41ff1,2347f47a,...,1,2,23143,320,50,2665,0,34,100034,221
1,13386437727020255392,0,14102705,1005,1,0a742914,510bd839,f028772b,ecad2386,7801e8d9,...,1,0,21412,320,50,2467,2,167,100077,23
2,18034954954610732886,1,14102817,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,22814,320,50,2647,2,35,100148,23
3,6676508794134781022,0,14102816,1005,0,85f751fd,c4e18dd6,50e219e0,39947756,2347f47a,...,1,3,21801,320,50,2515,1,161,100191,71
4,5503744330236741468,1,14102107,1005,1,5b4d2eda,16a36ef3,f028772b,ecad2386,7801e8d9,...,1,0,16208,320,50,1800,3,167,100077,23


In [6]:
train_1mm['click'].value_counts()

0    829759
1    170241
Name: click, dtype: int64

In [7]:
train_1mm.dtypes

id                  uint64
click                int64
hour                 int64
C1                   int64
banner_pos           int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type          int64
device_conn_type     int64
C14                  int64
C15                  int64
C16                  int64
C17                  int64
C18                  int64
C19                  int64
C20                  int64
C21                  int64
dtype: object

In [9]:
train_1mm.nunique()

id                  1000000
click                     2
hour                    216
C1                        7
banner_pos                7
site_id                2672
site_domain            2893
site_category            21
app_id                 3150
app_domain              210
app_category             28
device_id            151717
device_ip            546761
device_model           5181
device_type               5
device_conn_type          4
C14                    2117
C15                       8
C16                       9
C17                     397
C18                       4
C19                      65
C20                     162
C21                      55
dtype: int64

In [8]:
train_1mm['device_type'] = train_1mm['device_type'].astype(str)
train_1mm['device_conn_type'] = train_1mm['device_conn_type'].astype(str)
train_1mm['id'] = train_1mm['id'].astype(str)

val_50k['device_type'] = val_50k['device_type'].astype(str)
val_50k['device_conn_type'] = val_50k['device_conn_type'].astype(str)
val_50k['id'] = train_1mm['id'].astype(str)

In [10]:
val_50k['date'] = pd.to_datetime(val_50k['hour'], format='%y%m%d%H')
train_1mm['date'] = pd.to_datetime(train_1mm['hour'], format='%y%m%d%H')
train_1mm.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,date
0,4803607860744373534,0,14102809,1005,0,85f751fd,c4e18dd6,50e219e0,f0d41ff1,2347f47a,...,2,23143,320,50,2665,0,34,100034,221,2014-10-28 09:00:00
1,13386437727020255392,0,14102705,1005,1,0a742914,510bd839,f028772b,ecad2386,7801e8d9,...,0,21412,320,50,2467,2,167,100077,23,2014-10-27 05:00:00
2,18034954954610732886,1,14102817,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,0,22814,320,50,2647,2,35,100148,23,2014-10-28 17:00:00
3,6676508794134781022,0,14102816,1005,0,85f751fd,c4e18dd6,50e219e0,39947756,2347f47a,...,3,21801,320,50,2515,1,161,100191,71,2014-10-28 16:00:00
4,5503744330236741468,1,14102107,1005,1,5b4d2eda,16a36ef3,f028772b,ecad2386,7801e8d9,...,0,16208,320,50,1800,3,167,100077,23,2014-10-21 07:00:00


In [11]:
# date = '19073111'
# print(pd.to_datetime(date, format = '%y%m%d%H'))

In [14]:
train_1mm['year'] = train_1mm['date'].dt.year
train_1mm['day_of_year'] = train_1mm['date'].dt.dayofyear
train_1mm['weekday'] = train_1mm['date'].dt.weekday
train_1mm['week_of_year'] = train_1mm['date'].dt.week
train_1mm['day_of_month'] = train_1mm['date'].dt.day
train_1mm['quarter'] = train_1mm['date'].dt.quarter


val_50k['year'] = val_50k['date'].dt.year
val_50k['day_of_year'] = val_50k['date'].dt.dayofyear
val_50k['weekday'] = val_50k['date'].dt.weekday
val_50k['week_of_year'] = val_50k['date'].dt.week
val_50k['day_of_month'] = val_50k['date'].dt.day
val_50k['quarter'] = val_50k['date'].dt.quarter

In [15]:
train_1mm.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C19,C20,C21,date,year,day_of_year,weekday,week_of_year,day_of_month,quarter
0,4803607860744373534,0,14102809,1005,0,85f751fd,c4e18dd6,50e219e0,f0d41ff1,2347f47a,...,34,100034,221,2014-10-28 09:00:00,2014,301,1,44,28,4
1,13386437727020255392,0,14102705,1005,1,0a742914,510bd839,f028772b,ecad2386,7801e8d9,...,167,100077,23,2014-10-27 05:00:00,2014,300,0,44,27,4
2,18034954954610732886,1,14102817,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,35,100148,23,2014-10-28 17:00:00,2014,301,1,44,28,4
3,6676508794134781022,0,14102816,1005,0,85f751fd,c4e18dd6,50e219e0,39947756,2347f47a,...,161,100191,71,2014-10-28 16:00:00,2014,301,1,44,28,4
4,5503744330236741468,1,14102107,1005,1,5b4d2eda,16a36ef3,f028772b,ecad2386,7801e8d9,...,167,100077,23,2014-10-21 07:00:00,2014,294,1,43,21,4


In [16]:
X = train_1mm.drop(['click', 'date', 'hour', 'id', 'year', 'quarter'], axis=1)
y = train_1mm.click
categorical_feature_indices = np.where(X.dtypes == np.object)[0]

X2 = val_50k.drop(['click', 'date', 'hour', 'id', 'year', 'quarter'], axis=1)
y2 = val_50k.click

In [17]:
categorical_feature_indices

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [ ]:
#no id date, hour, 0.3924528516
#no date, hour, 0.3934399609
#no date, hour, year, quarter, 0.0.3923542578
#no date, hour, year, quarter, 0.0.3923542578 , scale_pos_weight = 0.2, 0.1487337382


# scale_pos_weight is num_neg/num_pos


clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Logloss')
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.5938043	test: 0.5914861	best: 0.5914861 (0)	total: 151ms	remaining: 2m 31s
1:	learn: 0.5322157	test: 0.5302578	best: 0.5302578 (1)	total: 284ms	remaining: 2m 21s
2:	learn: 0.4909670	test: 0.4889024	best: 0.4889024 (2)	total: 428ms	remaining: 2m 22s
3:	learn: 0.4640541	test: 0.4621085	best: 0.4621085 (3)	total: 571ms	remaining: 2m 22s
4:	learn: 0.4450696	test: 0.4425811	best: 0.4425811 (4)	total: 724ms	remaining: 2m 24s
5:	learn: 0.4332101	test: 0.4309393	best: 0.4309393 (5)	total: 857ms	remaining: 2m 21s
6:	learn: 0.4257307	test: 0.4235515	best: 0.4235515 (6)	total: 1.01s	remaining: 2m 23s
7:	learn: 0.4195233	test: 0.4172237	best: 0.4172237 (7)	total: 1.17s	remaining: 2m 25s
8:	learn: 0.4155631	test: 0.4132172	best: 0.4132172 (8)	total: 1.3s	remaining: 2m 23s
9:	learn: 0.4124714	test: 0.4101821	best: 0.4101821 (9)	total: 1.44s	remaining: 2m 22s
10:	learn: 0.4104545	test: 0.4083022	best: 0.4083022 (10)	total: 1.6s	remaining: 2m 24s
11:	learn: 0.

In [ ]:
proba_pred = clf.predict_proba(X2)
np.savetxt("catpred.csv", proba_pred, delimiter=",")

In [108]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Precision')
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.5279689	test: 0.5288270	best: 0.5288270 (0)	total: 155ms	remaining: 2m 34s
1:	learn: 0.6315861	test: 0.6351835	best: 0.6351835 (1)	total: 303ms	remaining: 2m 31s
2:	learn: 0.6336046	test: 0.6364658	best: 0.6364658 (2)	total: 435ms	remaining: 2m 24s
3:	learn: 0.6013464	test: 0.6032420	best: 0.6364658 (2)	total: 570ms	remaining: 2m 21s
4:	learn: 0.6095726	test: 0.6112140	best: 0.6364658 (2)	total: 719ms	remaining: 2m 23s
5:	learn: 0.6074301	test: 0.6096684	best: 0.6364658 (2)	total: 862ms	remaining: 2m 22s
6:	learn: 0.6135455	test: 0.6144535	best: 0.6364658 (2)	total: 1.03s	remaining: 2m 25s
7:	learn: 0.6020808	test: 0.6031157	best: 0.6364658 (2)	total: 1.19s	remaining: 2m 27s
8:	learn: 0.5984480	test: 0.6011135	best: 0.6364658 (2)	total: 1.35s	remaining: 2m 28s
9:	learn: 0.6000522	test: 0.6021891	best: 0.6364658 (2)	total: 1.5s	remaining: 2m 28s
10:	learn: 0.6006828	test: 0.6027906	best: 0.6364658 (2)	total: 1.65s	remaining: 2m 28s
11:	learn: 0.

94:	learn: 0.6171945	test: 0.6066982	best: 0.6364658 (2)	total: 12.5s	remaining: 1m 59s
95:	learn: 0.6172516	test: 0.6065761	best: 0.6364658 (2)	total: 12.7s	remaining: 1m 59s
96:	learn: 0.6174384	test: 0.6066351	best: 0.6364658 (2)	total: 12.8s	remaining: 1m 59s
97:	learn: 0.6176701	test: 0.6059995	best: 0.6364658 (2)	total: 12.9s	remaining: 1m 59s
98:	learn: 0.6178753	test: 0.6065948	best: 0.6364658 (2)	total: 13s	remaining: 1m 58s
99:	learn: 0.6183407	test: 0.6068856	best: 0.6364658 (2)	total: 13.2s	remaining: 1m 58s
100:	learn: 0.6184696	test: 0.6066982	best: 0.6364658 (2)	total: 13.3s	remaining: 1m 58s
101:	learn: 0.6186946	test: 0.6070182	best: 0.6364658 (2)	total: 13.4s	remaining: 1m 58s
102:	learn: 0.6185377	test: 0.6069652	best: 0.6364658 (2)	total: 13.6s	remaining: 1m 58s
103:	learn: 0.6186530	test: 0.6067844	best: 0.6364658 (2)	total: 13.7s	remaining: 1m 58s
104:	learn: 0.6187928	test: 0.6066834	best: 0.6364658 (2)	total: 13.8s	remaining: 1m 57s
105:	learn: 0.6191080	test: 0

187:	learn: 0.6288931	test: 0.6094134	best: 0.6364658 (2)	total: 24.8s	remaining: 1m 46s
188:	learn: 0.6291465	test: 0.6096030	best: 0.6364658 (2)	total: 24.9s	remaining: 1m 46s
189:	learn: 0.6290409	test: 0.6094559	best: 0.6364658 (2)	total: 25s	remaining: 1m 46s
190:	learn: 0.6294525	test: 0.6099550	best: 0.6364658 (2)	total: 25.1s	remaining: 1m 46s
191:	learn: 0.6293729	test: 0.6098224	best: 0.6364658 (2)	total: 25.3s	remaining: 1m 46s
192:	learn: 0.6295301	test: 0.6096670	best: 0.6364658 (2)	total: 25.4s	remaining: 1m 46s
193:	learn: 0.6289768	test: 0.6093272	best: 0.6364658 (2)	total: 25.6s	remaining: 1m 46s
194:	learn: 0.6289197	test: 0.6094373	best: 0.6364658 (2)	total: 25.7s	remaining: 1m 46s
195:	learn: 0.6287879	test: 0.6094875	best: 0.6364658 (2)	total: 25.8s	remaining: 1m 45s
196:	learn: 0.6293428	test: 0.6098696	best: 0.6364658 (2)	total: 26s	remaining: 1m 45s
197:	learn: 0.6293710	test: 0.6094915	best: 0.6364658 (2)	total: 26.1s	remaining: 1m 45s
198:	learn: 0.6291839	tes

281:	learn: 0.6376358	test: 0.6092545	best: 0.6364658 (2)	total: 37.2s	remaining: 1m 34s
282:	learn: 0.6378433	test: 0.6090706	best: 0.6364658 (2)	total: 37.4s	remaining: 1m 34s
283:	learn: 0.6381771	test: 0.6088579	best: 0.6364658 (2)	total: 37.5s	remaining: 1m 34s
284:	learn: 0.6379850	test: 0.6087786	best: 0.6364658 (2)	total: 37.6s	remaining: 1m 34s
285:	learn: 0.6384142	test: 0.6092842	best: 0.6364658 (2)	total: 37.8s	remaining: 1m 34s
286:	learn: 0.6380791	test: 0.6086416	best: 0.6364658 (2)	total: 37.9s	remaining: 1m 34s
287:	learn: 0.6375967	test: 0.6082919	best: 0.6364658 (2)	total: 38s	remaining: 1m 33s
288:	learn: 0.6377594	test: 0.6087498	best: 0.6364658 (2)	total: 38.1s	remaining: 1m 33s
289:	learn: 0.6379666	test: 0.6087173	best: 0.6364658 (2)	total: 38.3s	remaining: 1m 33s
290:	learn: 0.6382122	test: 0.6088870	best: 0.6364658 (2)	total: 38.4s	remaining: 1m 33s
291:	learn: 0.6380323	test: 0.6087137	best: 0.6364658 (2)	total: 38.5s	remaining: 1m 33s
292:	learn: 0.6376123	t

375:	learn: 0.6456500	test: 0.6060457	best: 0.6364658 (2)	total: 50.2s	remaining: 1m 23s
376:	learn: 0.6457639	test: 0.6057321	best: 0.6364658 (2)	total: 50.3s	remaining: 1m 23s
377:	learn: 0.6455255	test: 0.6064468	best: 0.6364658 (2)	total: 50.4s	remaining: 1m 23s
378:	learn: 0.6456190	test: 0.6062423	best: 0.6364658 (2)	total: 50.6s	remaining: 1m 22s
379:	learn: 0.6459989	test: 0.6068376	best: 0.6364658 (2)	total: 50.7s	remaining: 1m 22s
380:	learn: 0.6455716	test: 0.6059263	best: 0.6364658 (2)	total: 50.9s	remaining: 1m 22s
381:	learn: 0.6455112	test: 0.6058915	best: 0.6364658 (2)	total: 51s	remaining: 1m 22s
382:	learn: 0.6456885	test: 0.6058394	best: 0.6364658 (2)	total: 51.2s	remaining: 1m 22s
383:	learn: 0.6456734	test: 0.6057527	best: 0.6364658 (2)	total: 51.3s	remaining: 1m 22s
384:	learn: 0.6462034	test: 0.6064860	best: 0.6364658 (2)	total: 51.5s	remaining: 1m 22s
385:	learn: 0.6461578	test: 0.6065762	best: 0.6364658 (2)	total: 51.6s	remaining: 1m 22s
386:	learn: 0.6457793	t

...

563:	learn: 0.6516815	test: 0.6039895	best: 0.6364658 (2)	total: 1m 16s	remaining: 59.3s
564:	learn: 0.6514051	test: 0.6043554	best: 0.6364658 (2)	total: 1m 16s	remaining: 59.2s
565:	learn: 0.6512071	test: 0.6043108	best: 0.6364658 (2)	total: 1m 17s	remaining: 59.1s
566:	learn: 0.6513026	test: 0.6044515	best: 0.6364658 (2)	total: 1m 17s	remaining: 59s
567:	learn: 0.6512268	test: 0.6049009	best: 0.6364658 (2)	total: 1m 17s	remaining: 58.8s
568:	learn: 0.6515561	test: 0.6038590	best: 0.6364658 (2)	total: 1m 17s	remaining: 58.7s
569:	learn: 0.6516584	test: 0.6038755	best: 0.6364658 (2)	total: 1m 17s	remaining: 58.5s
570:	learn: 0.6516213	test: 0.6041766	best: 0.6364658 (2)	total: 1m 17s	remaining: 58.4s
571:	learn: 0.6516084	test: 0.6043502	best: 0.6364658 (2)	total: 1m 17s	remaining: 58.3s
572:	learn: 0.6517625	test: 0.6045274	best: 0.6364658 (2)	total: 1m 17s	remaining: 58.1s
573:	learn: 0.6519212	test: 0.6044183	best: 0.6364658 (2)	total: 1m 18s	remaining: 58s
574:	learn: 0.6517343	tes

657:	learn: 0.6535495	test: 0.6038720	best: 0.6364658 (2)	total: 1m 29s	remaining: 46.4s
658:	learn: 0.6536655	test: 0.6039573	best: 0.6364658 (2)	total: 1m 29s	remaining: 46.3s
659:	learn: 0.6538785	test: 0.6040629	best: 0.6364658 (2)	total: 1m 29s	remaining: 46.1s
660:	learn: 0.6541086	test: 0.6040958	best: 0.6364658 (2)	total: 1m 29s	remaining: 46s
661:	learn: 0.6541696	test: 0.6040497	best: 0.6364658 (2)	total: 1m 29s	remaining: 45.9s
662:	learn: 0.6539717	test: 0.6042408	best: 0.6364658 (2)	total: 1m 29s	remaining: 45.7s
663:	learn: 0.6537986	test: 0.6042045	best: 0.6364658 (2)	total: 1m 30s	remaining: 45.6s
664:	learn: 0.6536235	test: 0.6037467	best: 0.6364658 (2)	total: 1m 30s	remaining: 45.4s
665:	learn: 0.6534077	test: 0.6035652	best: 0.6364658 (2)	total: 1m 30s	remaining: 45.3s
666:	learn: 0.6534077	test: 0.6037066	best: 0.6364658 (2)	total: 1m 30s	remaining: 45.1s
667:	learn: 0.6534854	test: 0.6036128	best: 0.6364658 (2)	total: 1m 30s	remaining: 45s
668:	learn: 0.6534893	tes

751:	learn: 0.6577036	test: 0.6017360	best: 0.6364658 (2)	total: 1m 42s	remaining: 33.8s
752:	learn: 0.6577734	test: 0.6016876	best: 0.6364658 (2)	total: 1m 42s	remaining: 33.6s
753:	learn: 0.6577656	test: 0.6015961	best: 0.6364658 (2)	total: 1m 42s	remaining: 33.5s
754:	learn: 0.6578547	test: 0.6018613	best: 0.6364658 (2)	total: 1m 42s	remaining: 33.4s
755:	learn: 0.6577734	test: 0.6020967	best: 0.6364658 (2)	total: 1m 42s	remaining: 33.2s
756:	learn: 0.6577050	test: 0.6022220	best: 0.6364658 (2)	total: 1m 43s	remaining: 33.1s
757:	learn: 0.6577191	test: 0.6022843	best: 0.6364658 (2)	total: 1m 43s	remaining: 33s
758:	learn: 0.6579360	test: 0.6024096	best: 0.6364658 (2)	total: 1m 43s	remaining: 32.8s
759:	learn: 0.6580522	test: 0.6023597	best: 0.6364658 (2)	total: 1m 43s	remaining: 32.7s
760:	learn: 0.6580019	test: 0.6024068	best: 0.6364658 (2)	total: 1m 43s	remaining: 32.6s
761:	learn: 0.6578818	test: 0.6024680	best: 0.6364658 (2)	total: 1m 43s	remaining: 32.4s
762:	learn: 0.6575336	t

845:	learn: 0.6610219	test: 0.6017562	best: 0.6364658 (2)	total: 1m 54s	remaining: 20.9s
846:	learn: 0.6610674	test: 0.6014752	best: 0.6364658 (2)	total: 1m 54s	remaining: 20.8s
847:	learn: 0.6611171	test: 0.6011493	best: 0.6364658 (2)	total: 1m 55s	remaining: 20.6s
848:	learn: 0.6611344	test: 0.6012113	best: 0.6364658 (2)	total: 1m 55s	remaining: 20.5s
849:	learn: 0.6609574	test: 0.6011175	best: 0.6364658 (2)	total: 1m 55s	remaining: 20.4s
850:	learn: 0.6612352	test: 0.6011314	best: 0.6364658 (2)	total: 1m 55s	remaining: 20.2s
851:	learn: 0.6614769	test: 0.6010857	best: 0.6364658 (2)	total: 1m 55s	remaining: 20.1s
852:	learn: 0.6614373	test: 0.6015643	best: 0.6364658 (2)	total: 1m 55s	remaining: 19.9s
853:	learn: 0.6618379	test: 0.6015813	best: 0.6364658 (2)	total: 1m 55s	remaining: 19.8s
854:	learn: 0.6616090	test: 0.6011162	best: 0.6364658 (2)	total: 1m 56s	remaining: 19.7s
855:	learn: 0.6616908	test: 0.6012555	best: 0.6364658 (2)	total: 1m 56s	remaining: 19.5s
856:	learn: 0.6617410

940:	learn: 0.6648253	test: 0.6012199	best: 0.6364658 (2)	total: 2m 8s	remaining: 8.06s
941:	learn: 0.6648421	test: 0.6009723	best: 0.6364658 (2)	total: 2m 8s	remaining: 7.92s
942:	learn: 0.6650034	test: 0.6010958	best: 0.6364658 (2)	total: 2m 8s	remaining: 7.79s
943:	learn: 0.6649872	test: 0.6010642	best: 0.6364658 (2)	total: 2m 8s	remaining: 7.65s
944:	learn: 0.6651336	test: 0.6012185	best: 0.6364658 (2)	total: 2m 9s	remaining: 7.51s
945:	learn: 0.6654878	test: 0.6012038	best: 0.6364658 (2)	total: 2m 9s	remaining: 7.38s
946:	learn: 0.6651810	test: 0.6012648	best: 0.6364658 (2)	total: 2m 9s	remaining: 7.24s
947:	learn: 0.6653757	test: 0.6022070	best: 0.6364658 (2)	total: 2m 9s	remaining: 7.11s
948:	learn: 0.6651825	test: 0.6020369	best: 0.6364658 (2)	total: 2m 9s	remaining: 6.97s
949:	learn: 0.6651831	test: 0.6020046	best: 0.6364658 (2)	total: 2m 9s	remaining: 6.83s
950:	learn: 0.6650861	test: 0.6017576	best: 0.6364658 (2)	total: 2m 9s	remaining: 6.7s
951:	learn: 0.6650373	test: 0.602

In [109]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Recall')
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.0129593	test: 0.0125140	best: 0.0125140 (0)	total: 161ms	remaining: 2m 40s
1:	learn: 0.0340612	test: 0.0333784	best: 0.0333784 (1)	total: 307ms	remaining: 2m 33s
2:	learn: 0.0337235	test: 0.0332961	best: 0.0333784 (1)	total: 439ms	remaining: 2m 25s
3:	learn: 0.0603395	test: 0.0634637	best: 0.0634637 (3)	total: 574ms	remaining: 2m 22s
4:	learn: 0.0559194	test: 0.0582064	best: 0.0634637 (3)	total: 724ms	remaining: 2m 24s
5:	learn: 0.0582249	test: 0.0618524	best: 0.0634637 (3)	total: 867ms	remaining: 2m 23s
6:	learn: 0.0563379	test: 0.0597001	best: 0.0634637 (3)	total: 1.03s	remaining: 2m 26s
7:	learn: 0.0654351	test: 0.0687562	best: 0.0687562 (7)	total: 1.2s	remaining: 2m 28s
8:	learn: 0.0668155	test: 0.0711085	best: 0.0711085 (8)	total: 1.35s	remaining: 2m 29s
9:	learn: 0.0675497	test: 0.0711791	best: 0.0711791 (9)	total: 1.51s	remaining: 2m 29s
10:	learn: 0.0671752	test: 0.0706263	best: 0.0711791 (9)	total: 1.66s	remaining: 2m 29s
11:	learn: 0.

94:	learn: 0.0806044	test: 0.0807174	best: 0.0807174 (94)	total: 12.3s	remaining: 1m 57s
95:	learn: 0.0810376	test: 0.0812349	best: 0.0812349 (95)	total: 12.5s	remaining: 1m 57s
96:	learn: 0.0813240	test: 0.0813643	best: 0.0813643 (96)	total: 12.7s	remaining: 1m 57s
97:	learn: 0.0818747	test: 0.0821288	best: 0.0821288 (97)	total: 12.8s	remaining: 1m 58s
98:	learn: 0.0819848	test: 0.0822346	best: 0.0822346 (98)	total: 13s	remaining: 1m 58s
99:	learn: 0.0820802	test: 0.0822934	best: 0.0822934 (99)	total: 13.1s	remaining: 1m 57s
100:	learn: 0.0820802	test: 0.0822464	best: 0.0822934 (99)	total: 13.3s	remaining: 1m 57s
101:	learn: 0.0821096	test: 0.0822346	best: 0.0822934 (99)	total: 13.4s	remaining: 1m 57s
102:	learn: 0.0820802	test: 0.0824346	best: 0.0824346 (102)	total: 13.5s	remaining: 1m 57s
103:	learn: 0.0820288	test: 0.0824111	best: 0.0824346 (102)	total: 13.6s	remaining: 1m 57s
104:	learn: 0.0820949	test: 0.0826345	best: 0.0826345 (104)	total: 13.8s	remaining: 1m 57s
105:	learn: 0.0

185:	learn: 0.0855458	test: 0.0838812	best: 0.0846692 (174)	total: 24.4s	remaining: 1m 46s
186:	learn: 0.0856853	test: 0.0841164	best: 0.0846692 (174)	total: 24.6s	remaining: 1m 46s
187:	learn: 0.0858028	test: 0.0841517	best: 0.0846692 (174)	total: 24.7s	remaining: 1m 46s
188:	learn: 0.0857514	test: 0.0842105	best: 0.0846692 (174)	total: 24.8s	remaining: 1m 46s
189:	learn: 0.0858469	test: 0.0842693	best: 0.0846692 (174)	total: 25s	remaining: 1m 46s
190:	learn: 0.0860965	test: 0.0843634	best: 0.0846692 (174)	total: 25.1s	remaining: 1m 46s
191:	learn: 0.0860157	test: 0.0844693	best: 0.0846692 (174)	total: 25.2s	remaining: 1m 46s
192:	learn: 0.0860745	test: 0.0844693	best: 0.0846692 (174)	total: 25.4s	remaining: 1m 46s
193:	learn: 0.0860671	test: 0.0844222	best: 0.0846692 (174)	total: 25.5s	remaining: 1m 45s
194:	learn: 0.0861259	test: 0.0845869	best: 0.0846692 (174)	total: 25.6s	remaining: 1m 45s
195:	learn: 0.0862140	test: 0.0848221	best: 0.0848221 (195)	total: 25.7s	remaining: 1m 45s
1

277:	learn: 0.0887985	test: 0.0853278	best: 0.0858689 (230)	total: 36.6s	remaining: 1m 34s
278:	learn: 0.0888352	test: 0.0852220	best: 0.0858689 (230)	total: 36.7s	remaining: 1m 34s
279:	learn: 0.0888866	test: 0.0850809	best: 0.0858689 (230)	total: 36.8s	remaining: 1m 34s
280:	learn: 0.0889600	test: 0.0850456	best: 0.0858689 (230)	total: 37s	remaining: 1m 34s
281:	learn: 0.0889821	test: 0.0850456	best: 0.0858689 (230)	total: 37.1s	remaining: 1m 34s
282:	learn: 0.0896649	test: 0.0857160	best: 0.0858689 (230)	total: 37.3s	remaining: 1m 34s
283:	learn: 0.0896796	test: 0.0856454	best: 0.0858689 (230)	total: 37.4s	remaining: 1m 34s
284:	learn: 0.0896649	test: 0.0856807	best: 0.0858689 (230)	total: 37.5s	remaining: 1m 34s
285:	learn: 0.0897310	test: 0.0855043	best: 0.0858689 (230)	total: 37.7s	remaining: 1m 34s
286:	learn: 0.0897236	test: 0.0855513	best: 0.0858689 (230)	total: 37.8s	remaining: 1m 33s
287:	learn: 0.0897897	test: 0.0853278	best: 0.0858689 (230)	total: 38s	remaining: 1m 33s
288

369:	learn: 0.0910673	test: 0.0864452	best: 0.0867745 (361)	total: 48.8s	remaining: 1m 23s
370:	learn: 0.0911554	test: 0.0865157	best: 0.0867745 (361)	total: 48.9s	remaining: 1m 22s
371:	learn: 0.0911554	test: 0.0865510	best: 0.0867745 (361)	total: 49.1s	remaining: 1m 22s
372:	learn: 0.0911994	test: 0.0863628	best: 0.0867745 (361)	total: 49.2s	remaining: 1m 22s
373:	learn: 0.0910967	test: 0.0863158	best: 0.0867745 (361)	total: 49.3s	remaining: 1m 22s
374:	learn: 0.0911481	test: 0.0863276	best: 0.0867745 (361)	total: 49.4s	remaining: 1m 22s
375:	learn: 0.0911921	test: 0.0862805	best: 0.0867745 (361)	total: 49.6s	remaining: 1m 22s
376:	learn: 0.0912362	test: 0.0862805	best: 0.0867745 (361)	total: 49.7s	remaining: 1m 22s
377:	learn: 0.0914271	test: 0.0865157	best: 0.0867745 (361)	total: 49.8s	remaining: 1m 21s
378:	learn: 0.0914197	test: 0.0865393	best: 0.0867745 (361)	total: 49.9s	remaining: 1m 21s
379:	learn: 0.0914785	test: 0.0865745	best: 0.0867745 (361)	total: 50.1s	remaining: 1m 21s

...

553:	learn: 0.0957297	test: 0.0903264	best: 0.0903264 (553)	total: 1m 13s	remaining: 59.3s
554:	learn: 0.0957444	test: 0.0903381	best: 0.0903381 (554)	total: 1m 13s	remaining: 59.2s
555:	learn: 0.0958325	test: 0.0905146	best: 0.0905146 (555)	total: 1m 13s	remaining: 59s
556:	learn: 0.0958472	test: 0.0905381	best: 0.0905381 (556)	total: 1m 14s	remaining: 58.9s
557:	learn: 0.0958325	test: 0.0904910	best: 0.0905381 (556)	total: 1m 14s	remaining: 58.8s
558:	learn: 0.0958765	test: 0.0905498	best: 0.0905498 (558)	total: 1m 14s	remaining: 58.6s
559:	learn: 0.0960307	test: 0.0905616	best: 0.0905616 (559)	total: 1m 14s	remaining: 58.5s
560:	learn: 0.0960454	test: 0.0905616	best: 0.0905616 (559)	total: 1m 14s	remaining: 58.4s
561:	learn: 0.0959940	test: 0.0905734	best: 0.0905734 (561)	total: 1m 14s	remaining: 58.3s
562:	learn: 0.0959940	test: 0.0905616	best: 0.0905734 (561)	total: 1m 14s	remaining: 58.1s
563:	learn: 0.0960014	test: 0.0905851	best: 0.0905851 (563)	total: 1m 15s	remaining: 58s
564

645:	learn: 0.0971174	test: 0.0902558	best: 0.0907615 (596)	total: 1m 26s	remaining: 47.1s
646:	learn: 0.0971468	test: 0.0903029	best: 0.0907615 (596)	total: 1m 26s	remaining: 47s
647:	learn: 0.0973377	test: 0.0905381	best: 0.0907615 (596)	total: 1m 26s	remaining: 46.8s
648:	learn: 0.0976314	test: 0.0907851	best: 0.0907851 (648)	total: 1m 26s	remaining: 46.7s
649:	learn: 0.0976167	test: 0.0907968	best: 0.0907968 (649)	total: 1m 26s	remaining: 46.6s
650:	learn: 0.0976240	test: 0.0908321	best: 0.0908321 (650)	total: 1m 26s	remaining: 46.5s
651:	learn: 0.0975946	test: 0.0907968	best: 0.0908321 (650)	total: 1m 26s	remaining: 46.3s
652:	learn: 0.0976534	test: 0.0908909	best: 0.0908909 (652)	total: 1m 26s	remaining: 46.2s
653:	learn: 0.0975946	test: 0.0909262	best: 0.0909262 (653)	total: 1m 27s	remaining: 46.1s
654:	learn: 0.0976314	test: 0.0910673	best: 0.0910673 (654)	total: 1m 27s	remaining: 45.9s
655:	learn: 0.0976681	test: 0.0910673	best: 0.0910673 (654)	total: 1m 27s	remaining: 45.8s
6

737:	learn: 0.0982187	test: 0.0909732	best: 0.0912085 (691)	total: 1m 38s	remaining: 35s
738:	learn: 0.0982334	test: 0.0909615	best: 0.0912085 (691)	total: 1m 38s	remaining: 34.9s
739:	learn: 0.0983142	test: 0.0909850	best: 0.0912085 (691)	total: 1m 38s	remaining: 34.8s
740:	learn: 0.0983142	test: 0.0909497	best: 0.0912085 (691)	total: 1m 39s	remaining: 34.6s
741:	learn: 0.0983436	test: 0.0909850	best: 0.0912085 (691)	total: 1m 39s	remaining: 34.5s
742:	learn: 0.0983656	test: 0.0909380	best: 0.0912085 (691)	total: 1m 39s	remaining: 34.4s
743:	learn: 0.0983582	test: 0.0909968	best: 0.0912085 (691)	total: 1m 39s	remaining: 34.2s
744:	learn: 0.0984390	test: 0.0910556	best: 0.0912085 (691)	total: 1m 39s	remaining: 34.1s
745:	learn: 0.0984096	test: 0.0910556	best: 0.0912085 (691)	total: 1m 39s	remaining: 34s
746:	learn: 0.0984757	test: 0.0911144	best: 0.0912085 (691)	total: 1m 39s	remaining: 33.8s
747:	learn: 0.0984537	test: 0.0911261	best: 0.0912085 (691)	total: 1m 40s	remaining: 33.7s
748

829:	learn: 0.0996578	test: 0.0915495	best: 0.0916201 (821)	total: 1m 50s	remaining: 22.6s
830:	learn: 0.0996946	test: 0.0914555	best: 0.0916201 (821)	total: 1m 50s	remaining: 22.5s
831:	learn: 0.0996358	test: 0.0913849	best: 0.0916201 (821)	total: 1m 50s	remaining: 22.4s
832:	learn: 0.0996505	test: 0.0914319	best: 0.0916201 (821)	total: 1m 50s	remaining: 22.2s
833:	learn: 0.0998414	test: 0.0915848	best: 0.0916201 (821)	total: 1m 51s	remaining: 22.1s
834:	learn: 0.0998487	test: 0.0915966	best: 0.0916201 (821)	total: 1m 51s	remaining: 22s
835:	learn: 0.0999001	test: 0.0916554	best: 0.0916554 (835)	total: 1m 51s	remaining: 21.9s
836:	learn: 0.0999001	test: 0.0915966	best: 0.0916554 (835)	total: 1m 51s	remaining: 21.7s
837:	learn: 0.0999001	test: 0.0915143	best: 0.0916554 (835)	total: 1m 51s	remaining: 21.6s
838:	learn: 0.0998928	test: 0.0915025	best: 0.0916554 (835)	total: 1m 51s	remaining: 21.4s
839:	learn: 0.0999222	test: 0.0915025	best: 0.0916554 (835)	total: 1m 51s	remaining: 21.3s
8

920:	learn: 0.1010969	test: 0.0925022	best: 0.0925257 (918)	total: 2m 3s	remaining: 10.6s
921:	learn: 0.1010676	test: 0.0925963	best: 0.0925963 (921)	total: 2m 3s	remaining: 10.4s
922:	learn: 0.1010823	test: 0.0926198	best: 0.0926198 (922)	total: 2m 3s	remaining: 10.3s
923:	learn: 0.1011263	test: 0.0925728	best: 0.0926198 (922)	total: 2m 3s	remaining: 10.2s
924:	learn: 0.1010896	test: 0.0925963	best: 0.0926198 (922)	total: 2m 3s	remaining: 10s
925:	learn: 0.1011043	test: 0.0925963	best: 0.0926198 (922)	total: 2m 4s	remaining: 9.91s
926:	learn: 0.1011043	test: 0.0925845	best: 0.0926198 (922)	total: 2m 4s	remaining: 9.78s
927:	learn: 0.1011043	test: 0.0926198	best: 0.0926198 (922)	total: 2m 4s	remaining: 9.65s
928:	learn: 0.1011337	test: 0.0927139	best: 0.0927139 (928)	total: 2m 4s	remaining: 9.51s
929:	learn: 0.1011410	test: 0.0927610	best: 0.0927610 (929)	total: 2m 4s	remaining: 9.38s
930:	learn: 0.1011704	test: 0.0928198	best: 0.0928198 (930)	total: 2m 4s	remaining: 9.25s
931:	learn: 

In [111]:
catpred = clf.predict(X2)
np.savetxt("catpred.csv", catpred, delimiter=",")

In [42]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Logloss', scale_pos_weight = 4.874)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.6663182	test: 0.6655904	best: 0.6655904 (0)	total: 151ms	remaining: 2m 31s
1:	learn: 0.6480076	test: 0.6466006	best: 0.6466006 (1)	total: 324ms	remaining: 2m 41s
2:	learn: 0.6350584	test: 0.6332875	best: 0.6332875 (2)	total: 484ms	remaining: 2m 40s
3:	learn: 0.6258205	test: 0.6241241	best: 0.6241241 (3)	total: 625ms	remaining: 2m 35s
4:	learn: 0.6191749	test: 0.6173931	best: 0.6173931 (4)	total: 789ms	remaining: 2m 36s
5:	learn: 0.6144669	test: 0.6124855	best: 0.6124855 (5)	total: 933ms	remaining: 2m 34s
6:	learn: 0.6108735	test: 0.6092503	best: 0.6092503 (6)	total: 1.09s	remaining: 2m 34s
7:	learn: 0.6079225	test: 0.6059914	best: 0.6059914 (7)	total: 1.23s	remaining: 2m 32s
8:	learn: 0.6057963	test: 0.6039926	best: 0.6039926 (8)	total: 1.37s	remaining: 2m 30s
9:	learn: 0.6043186	test: 0.6024545	best: 0.6024545 (9)	total: 1.51s	remaining: 2m 29s
10:	learn: 0.6029936	test: 0.6011312	best: 0.6011312 (10)	total: 1.68s	remaining: 2m 31s
11:	learn: 

93:	learn: 0.5873602	test: 0.5870794	best: 0.5870794 (93)	total: 13.7s	remaining: 2m 12s
94:	learn: 0.5872899	test: 0.5870020	best: 0.5870020 (94)	total: 13.9s	remaining: 2m 11s
95:	learn: 0.5872280	test: 0.5869555	best: 0.5869555 (95)	total: 14s	remaining: 2m 11s
96:	learn: 0.5871482	test: 0.5868872	best: 0.5868872 (96)	total: 14.1s	remaining: 2m 11s
97:	learn: 0.5870729	test: 0.5868358	best: 0.5868358 (97)	total: 14.2s	remaining: 2m 11s
98:	learn: 0.5870298	test: 0.5868010	best: 0.5868010 (98)	total: 14.4s	remaining: 2m 11s
99:	learn: 0.5869219	test: 0.5867042	best: 0.5867042 (99)	total: 14.5s	remaining: 2m 10s
100:	learn: 0.5868697	test: 0.5866730	best: 0.5866730 (100)	total: 14.7s	remaining: 2m 10s
101:	learn: 0.5867932	test: 0.5866368	best: 0.5866368 (101)	total: 14.8s	remaining: 2m 10s
102:	learn: 0.5867528	test: 0.5866120	best: 0.5866120 (102)	total: 15s	remaining: 2m 10s
103:	learn: 0.5867186	test: 0.5865767	best: 0.5865767 (103)	total: 15.1s	remaining: 2m 10s
104:	learn: 0.586

185:	learn: 0.5829863	test: 0.5845264	best: 0.5845264 (185)	total: 26.1s	remaining: 1m 54s
186:	learn: 0.5829474	test: 0.5845010	best: 0.5845010 (186)	total: 26.2s	remaining: 1m 54s
187:	learn: 0.5829016	test: 0.5844658	best: 0.5844658 (187)	total: 26.4s	remaining: 1m 54s
188:	learn: 0.5828723	test: 0.5844631	best: 0.5844631 (188)	total: 26.5s	remaining: 1m 53s
189:	learn: 0.5828602	test: 0.5844650	best: 0.5844631 (188)	total: 26.6s	remaining: 1m 53s
190:	learn: 0.5828074	test: 0.5844250	best: 0.5844250 (190)	total: 26.8s	remaining: 1m 53s
191:	learn: 0.5827498	test: 0.5843918	best: 0.5843918 (191)	total: 26.9s	remaining: 1m 53s
192:	learn: 0.5827050	test: 0.5843674	best: 0.5843674 (192)	total: 27s	remaining: 1m 53s
193:	learn: 0.5826635	test: 0.5843434	best: 0.5843434 (193)	total: 27.2s	remaining: 1m 52s
194:	learn: 0.5826276	test: 0.5843266	best: 0.5843266 (194)	total: 27.3s	remaining: 1m 52s
195:	learn: 0.5826017	test: 0.5843130	best: 0.5843130 (195)	total: 27.4s	remaining: 1m 52s
1

277:	learn: 0.5802241	test: 0.5833830	best: 0.5833830 (277)	total: 38.2s	remaining: 1m 39s
278:	learn: 0.5801889	test: 0.5833777	best: 0.5833777 (278)	total: 38.4s	remaining: 1m 39s
279:	learn: 0.5801630	test: 0.5833803	best: 0.5833777 (278)	total: 38.5s	remaining: 1m 38s
280:	learn: 0.5801263	test: 0.5833587	best: 0.5833587 (280)	total: 38.6s	remaining: 1m 38s
281:	learn: 0.5801062	test: 0.5833600	best: 0.5833587 (280)	total: 38.7s	remaining: 1m 38s
282:	learn: 0.5800887	test: 0.5833438	best: 0.5833438 (282)	total: 38.9s	remaining: 1m 38s
283:	learn: 0.5800692	test: 0.5833431	best: 0.5833431 (283)	total: 39s	remaining: 1m 38s
284:	learn: 0.5800447	test: 0.5833352	best: 0.5833352 (284)	total: 39.2s	remaining: 1m 38s
285:	learn: 0.5800334	test: 0.5833313	best: 0.5833313 (285)	total: 39.3s	remaining: 1m 38s
286:	learn: 0.5800088	test: 0.5833194	best: 0.5833194 (286)	total: 39.4s	remaining: 1m 37s
287:	learn: 0.5799879	test: 0.5833189	best: 0.5833189 (287)	total: 39.5s	remaining: 1m 37s
2

369:	learn: 0.5779796	test: 0.5827834	best: 0.5827834 (368)	total: 50.5s	remaining: 1m 25s
370:	learn: 0.5779411	test: 0.5827649	best: 0.5827649 (370)	total: 50.7s	remaining: 1m 25s
371:	learn: 0.5779207	test: 0.5827612	best: 0.5827612 (371)	total: 50.8s	remaining: 1m 25s
372:	learn: 0.5778999	test: 0.5827536	best: 0.5827536 (372)	total: 50.9s	remaining: 1m 25s
373:	learn: 0.5778551	test: 0.5827287	best: 0.5827287 (373)	total: 51s	remaining: 1m 25s
374:	learn: 0.5778154	test: 0.5827019	best: 0.5827019 (374)	total: 51.1s	remaining: 1m 25s
375:	learn: 0.5778112	test: 0.5827019	best: 0.5827019 (374)	total: 51.3s	remaining: 1m 25s
376:	learn: 0.5777831	test: 0.5826943	best: 0.5826943 (376)	total: 51.4s	remaining: 1m 24s
377:	learn: 0.5777555	test: 0.5826917	best: 0.5826917 (377)	total: 51.6s	remaining: 1m 24s
378:	learn: 0.5777355	test: 0.5827017	best: 0.5826917 (377)	total: 51.7s	remaining: 1m 24s
379:	learn: 0.5777208	test: 0.5826936	best: 0.5826917 (377)	total: 51.8s	remaining: 1m 24s
3

...

553:	learn: 0.5741478	test: 0.5821929	best: 0.5821929 (553)	total: 1m 14s	remaining: 1m
554:	learn: 0.5741282	test: 0.5821844	best: 0.5821844 (554)	total: 1m 14s	remaining: 60s
555:	learn: 0.5741048	test: 0.5821776	best: 0.5821776 (555)	total: 1m 14s	remaining: 59.8s
556:	learn: 0.5740738	test: 0.5821799	best: 0.5821776 (555)	total: 1m 15s	remaining: 59.7s
557:	learn: 0.5740548	test: 0.5821823	best: 0.5821776 (555)	total: 1m 15s	remaining: 59.6s
558:	learn: 0.5740281	test: 0.5821750	best: 0.5821750 (558)	total: 1m 15s	remaining: 59.5s
559:	learn: 0.5739985	test: 0.5821797	best: 0.5821750 (558)	total: 1m 15s	remaining: 59.3s
560:	learn: 0.5739830	test: 0.5821804	best: 0.5821750 (558)	total: 1m 15s	remaining: 59.2s
561:	learn: 0.5739673	test: 0.5821731	best: 0.5821731 (561)	total: 1m 15s	remaining: 59.1s
562:	learn: 0.5739505	test: 0.5821636	best: 0.5821636 (562)	total: 1m 15s	remaining: 58.9s
563:	learn: 0.5739362	test: 0.5821677	best: 0.5821636 (562)	total: 1m 16s	remaining: 58.8s
564:

644:	learn: 0.5724498	test: 0.5819792	best: 0.5819487 (624)	total: 1m 26s	remaining: 47.7s
645:	learn: 0.5724386	test: 0.5819834	best: 0.5819487 (624)	total: 1m 26s	remaining: 47.6s
646:	learn: 0.5724278	test: 0.5819822	best: 0.5819487 (624)	total: 1m 26s	remaining: 47.4s
647:	learn: 0.5724109	test: 0.5819885	best: 0.5819487 (624)	total: 1m 27s	remaining: 47.3s
648:	learn: 0.5723851	test: 0.5819867	best: 0.5819487 (624)	total: 1m 27s	remaining: 47.2s
649:	learn: 0.5723669	test: 0.5819848	best: 0.5819487 (624)	total: 1m 27s	remaining: 47s
650:	learn: 0.5723277	test: 0.5819825	best: 0.5819487 (624)	total: 1m 27s	remaining: 46.9s
651:	learn: 0.5723102	test: 0.5819864	best: 0.5819487 (624)	total: 1m 27s	remaining: 46.8s
652:	learn: 0.5722969	test: 0.5819887	best: 0.5819487 (624)	total: 1m 27s	remaining: 46.6s
653:	learn: 0.5722722	test: 0.5819854	best: 0.5819487 (624)	total: 1m 27s	remaining: 46.5s
654:	learn: 0.5722550	test: 0.5819863	best: 0.5819487 (624)	total: 1m 28s	remaining: 46.4s
6

736:	learn: 0.5709212	test: 0.5819209	best: 0.5818911 (731)	total: 1m 38s	remaining: 35.3s
737:	learn: 0.5709037	test: 0.5819202	best: 0.5818911 (731)	total: 1m 39s	remaining: 35.2s
738:	learn: 0.5708923	test: 0.5819018	best: 0.5818911 (731)	total: 1m 39s	remaining: 35s
739:	learn: 0.5708757	test: 0.5818994	best: 0.5818911 (731)	total: 1m 39s	remaining: 34.9s
740:	learn: 0.5708606	test: 0.5818920	best: 0.5818911 (731)	total: 1m 39s	remaining: 34.7s
741:	learn: 0.5708440	test: 0.5818927	best: 0.5818911 (731)	total: 1m 39s	remaining: 34.6s
742:	learn: 0.5708227	test: 0.5818902	best: 0.5818902 (742)	total: 1m 39s	remaining: 34.5s
743:	learn: 0.5708083	test: 0.5818836	best: 0.5818836 (743)	total: 1m 39s	remaining: 34.4s
744:	learn: 0.5707879	test: 0.5818802	best: 0.5818802 (744)	total: 1m 39s	remaining: 34.2s
745:	learn: 0.5707719	test: 0.5818809	best: 0.5818802 (744)	total: 1m 40s	remaining: 34.1s
746:	learn: 0.5707564	test: 0.5818856	best: 0.5818802 (744)	total: 1m 40s	remaining: 33.9s
7

828:	learn: 0.5694762	test: 0.5818881	best: 0.5818351 (815)	total: 1m 51s	remaining: 22.9s
829:	learn: 0.5694612	test: 0.5818843	best: 0.5818351 (815)	total: 1m 51s	remaining: 22.8s
830:	learn: 0.5694390	test: 0.5818828	best: 0.5818351 (815)	total: 1m 51s	remaining: 22.7s
831:	learn: 0.5694287	test: 0.5818862	best: 0.5818351 (815)	total: 1m 51s	remaining: 22.5s
832:	learn: 0.5694041	test: 0.5818863	best: 0.5818351 (815)	total: 1m 51s	remaining: 22.4s
833:	learn: 0.5693804	test: 0.5818723	best: 0.5818351 (815)	total: 1m 51s	remaining: 22.3s
834:	learn: 0.5693669	test: 0.5818776	best: 0.5818351 (815)	total: 1m 52s	remaining: 22.1s
835:	learn: 0.5693590	test: 0.5818783	best: 0.5818351 (815)	total: 1m 52s	remaining: 22s
836:	learn: 0.5693325	test: 0.5818780	best: 0.5818351 (815)	total: 1m 52s	remaining: 21.9s
837:	learn: 0.5693108	test: 0.5818928	best: 0.5818351 (815)	total: 1m 52s	remaining: 21.7s
838:	learn: 0.5692855	test: 0.5818717	best: 0.5818351 (815)	total: 1m 52s	remaining: 21.6s
8

920:	learn: 0.5680254	test: 0.5818306	best: 0.5818306 (920)	total: 2m 3s	remaining: 10.6s
921:	learn: 0.5679951	test: 0.5818195	best: 0.5818195 (921)	total: 2m 3s	remaining: 10.4s
922:	learn: 0.5679824	test: 0.5818211	best: 0.5818195 (921)	total: 2m 3s	remaining: 10.3s
923:	learn: 0.5679683	test: 0.5818235	best: 0.5818195 (921)	total: 2m 3s	remaining: 10.2s
924:	learn: 0.5679489	test: 0.5818231	best: 0.5818195 (921)	total: 2m 3s	remaining: 10s
925:	learn: 0.5679236	test: 0.5818126	best: 0.5818126 (925)	total: 2m 4s	remaining: 9.91s
926:	learn: 0.5679073	test: 0.5818113	best: 0.5818113 (926)	total: 2m 4s	remaining: 9.78s
927:	learn: 0.5678798	test: 0.5818003	best: 0.5818003 (927)	total: 2m 4s	remaining: 9.65s
928:	learn: 0.5678648	test: 0.5818087	best: 0.5818003 (927)	total: 2m 4s	remaining: 9.52s
929:	learn: 0.5678455	test: 0.5818088	best: 0.5818003 (927)	total: 2m 4s	remaining: 9.38s
930:	learn: 0.5678263	test: 0.5818111	best: 0.5818003 (927)	total: 2m 4s	remaining: 9.24s
931:	learn: 

In [113]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Precision', scale_pos_weight = 4.874)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.6613203	test: 0.6612318	best: 0.6612318 (0)	total: 171ms	remaining: 2m 50s
1:	learn: 0.6496409	test: 0.6511300	best: 0.6612318 (0)	total: 345ms	remaining: 2m 52s
2:	learn: 0.6495983	test: 0.6511220	best: 0.6612318 (0)	total: 504ms	remaining: 2m 47s
3:	learn: 0.6480773	test: 0.6507760	best: 0.6612318 (0)	total: 678ms	remaining: 2m 48s
4:	learn: 0.6465364	test: 0.6490315	best: 0.6612318 (0)	total: 829ms	remaining: 2m 44s
5:	learn: 0.6471766	test: 0.6493527	best: 0.6612318 (0)	total: 976ms	remaining: 2m 41s
6:	learn: 0.6477397	test: 0.6501576	best: 0.6612318 (0)	total: 1.16s	remaining: 2m 44s
7:	learn: 0.6490617	test: 0.6513621	best: 0.6612318 (0)	total: 1.34s	remaining: 2m 46s
8:	learn: 0.6497961	test: 0.6520352	best: 0.6612318 (0)	total: 1.51s	remaining: 2m 46s
9:	learn: 0.6495607	test: 0.6522547	best: 0.6612318 (0)	total: 1.67s	remaining: 2m 44s
10:	learn: 0.6494565	test: 0.6519937	best: 0.6612318 (0)	total: 1.79s	remaining: 2m 41s
11:	learn: 0

95:	learn: 0.6604365	test: 0.6591829	best: 0.6612318 (0)	total: 13.2s	remaining: 2m 3s
96:	learn: 0.6605225	test: 0.6591790	best: 0.6612318 (0)	total: 13.3s	remaining: 2m 3s
97:	learn: 0.6607058	test: 0.6593316	best: 0.6612318 (0)	total: 13.4s	remaining: 2m 3s
98:	learn: 0.6607106	test: 0.6592593	best: 0.6612318 (0)	total: 13.6s	remaining: 2m 3s
99:	learn: 0.6607582	test: 0.6592715	best: 0.6612318 (0)	total: 13.7s	remaining: 2m 3s
100:	learn: 0.6609100	test: 0.6594560	best: 0.6612318 (0)	total: 13.8s	remaining: 2m 2s
101:	learn: 0.6610238	test: 0.6595318	best: 0.6612318 (0)	total: 14s	remaining: 2m 2s
102:	learn: 0.6610723	test: 0.6595683	best: 0.6612318 (0)	total: 14.1s	remaining: 2m 3s
103:	learn: 0.6611853	test: 0.6596798	best: 0.6612318 (0)	total: 14.3s	remaining: 2m 2s
104:	learn: 0.6613065	test: 0.6597503	best: 0.6612318 (0)	total: 14.4s	remaining: 2m 3s
105:	learn: 0.6616504	test: 0.6599286	best: 0.6612318 (0)	total: 14.6s	remaining: 2m 2s
106:	learn: 0.6616698	test: 0.6599059	b

189:	learn: 0.6642883	test: 0.6615681	best: 0.6616589 (182)	total: 26.3s	remaining: 1m 52s
190:	learn: 0.6643508	test: 0.6615370	best: 0.6616589 (182)	total: 26.4s	remaining: 1m 51s
191:	learn: 0.6643594	test: 0.6615783	best: 0.6616589 (182)	total: 26.5s	remaining: 1m 51s
192:	learn: 0.6643779	test: 0.6615728	best: 0.6616589 (182)	total: 26.7s	remaining: 1m 51s
193:	learn: 0.6643330	test: 0.6616535	best: 0.6616589 (182)	total: 26.8s	remaining: 1m 51s
194:	learn: 0.6643541	test: 0.6616066	best: 0.6616589 (182)	total: 27s	remaining: 1m 51s
195:	learn: 0.6643580	test: 0.6615548	best: 0.6616589 (182)	total: 27.1s	remaining: 1m 51s
196:	learn: 0.6643613	test: 0.6615700	best: 0.6616589 (182)	total: 27.2s	remaining: 1m 50s
197:	learn: 0.6643879	test: 0.6615251	best: 0.6616589 (182)	total: 27.4s	remaining: 1m 50s
198:	learn: 0.6644266	test: 0.6615404	best: 0.6616589 (182)	total: 27.5s	remaining: 1m 50s
199:	learn: 0.6644244	test: 0.6615052	best: 0.6616589 (182)	total: 27.6s	remaining: 1m 50s
2

281:	learn: 0.6664334	test: 0.6624164	best: 0.6624919 (267)	total: 38.7s	remaining: 1m 38s
282:	learn: 0.6664745	test: 0.6624310	best: 0.6624919 (267)	total: 38.8s	remaining: 1m 38s
283:	learn: 0.6664836	test: 0.6624753	best: 0.6624919 (267)	total: 38.9s	remaining: 1m 38s
284:	learn: 0.6664679	test: 0.6624574	best: 0.6624919 (267)	total: 39.1s	remaining: 1m 38s
285:	learn: 0.6665054	test: 0.6624788	best: 0.6624919 (267)	total: 39.3s	remaining: 1m 37s
286:	learn: 0.6665084	test: 0.6624914	best: 0.6624919 (267)	total: 39.4s	remaining: 1m 37s
287:	learn: 0.6665095	test: 0.6625094	best: 0.6625094 (287)	total: 39.5s	remaining: 1m 37s
288:	learn: 0.6665406	test: 0.6625598	best: 0.6625598 (288)	total: 39.6s	remaining: 1m 37s
289:	learn: 0.6665309	test: 0.6625265	best: 0.6625598 (288)	total: 39.7s	remaining: 1m 37s
290:	learn: 0.6665910	test: 0.6624716	best: 0.6625598 (288)	total: 39.9s	remaining: 1m 37s
291:	learn: 0.6665968	test: 0.6624994	best: 0.6625598 (288)	total: 40s	remaining: 1m 37s
2

372:	learn: 0.6679802	test: 0.6632671	best: 0.6632856 (366)	total: 50.9s	remaining: 1m 25s
373:	learn: 0.6679805	test: 0.6632864	best: 0.6632864 (373)	total: 51s	remaining: 1m 25s
374:	learn: 0.6680112	test: 0.6633155	best: 0.6633155 (374)	total: 51.2s	remaining: 1m 25s
375:	learn: 0.6680236	test: 0.6634042	best: 0.6634042 (375)	total: 51.3s	remaining: 1m 25s
376:	learn: 0.6680729	test: 0.6634291	best: 0.6634291 (376)	total: 51.5s	remaining: 1m 25s
377:	learn: 0.6681110	test: 0.6635005	best: 0.6635005 (377)	total: 51.6s	remaining: 1m 24s
378:	learn: 0.6681572	test: 0.6635277	best: 0.6635277 (378)	total: 51.8s	remaining: 1m 24s
379:	learn: 0.6681640	test: 0.6635176	best: 0.6635277 (378)	total: 51.9s	remaining: 1m 24s
380:	learn: 0.6682033	test: 0.6635419	best: 0.6635419 (380)	total: 52s	remaining: 1m 24s
381:	learn: 0.6682542	test: 0.6635904	best: 0.6635904 (381)	total: 52.2s	remaining: 1m 24s
382:	learn: 0.6682081	test: 0.6635812	best: 0.6635904 (381)	total: 52.3s	remaining: 1m 24s
383

...

555:	learn: 0.6705675	test: 0.6643175	best: 0.6643754 (546)	total: 1m 15s	remaining: 1m
556:	learn: 0.6705758	test: 0.6643170	best: 0.6643754 (546)	total: 1m 15s	remaining: 1m
557:	learn: 0.6705732	test: 0.6643009	best: 0.6643754 (546)	total: 1m 15s	remaining: 60s
558:	learn: 0.6705859	test: 0.6642713	best: 0.6643754 (546)	total: 1m 15s	remaining: 59.8s
559:	learn: 0.6706015	test: 0.6642868	best: 0.6643754 (546)	total: 1m 15s	remaining: 59.7s
560:	learn: 0.6706373	test: 0.6643227	best: 0.6643754 (546)	total: 1m 16s	remaining: 59.6s
561:	learn: 0.6707000	test: 0.6643727	best: 0.6643754 (546)	total: 1m 16s	remaining: 59.4s
562:	learn: 0.6706981	test: 0.6643787	best: 0.6643787 (562)	total: 1m 16s	remaining: 59.3s
563:	learn: 0.6706924	test: 0.6643654	best: 0.6643787 (562)	total: 1m 16s	remaining: 59.2s
564:	learn: 0.6707063	test: 0.6643696	best: 0.6643787 (562)	total: 1m 16s	remaining: 59s
565:	learn: 0.6707201	test: 0.6644283	best: 0.6644283 (565)	total: 1m 16s	remaining: 58.9s
566:	lear

647:	learn: 0.6717791	test: 0.6650423	best: 0.6650423 (647)	total: 1m 27s	remaining: 47.7s
648:	learn: 0.6718030	test: 0.6650257	best: 0.6650423 (647)	total: 1m 28s	remaining: 47.6s
649:	learn: 0.6718243	test: 0.6650625	best: 0.6650625 (649)	total: 1m 28s	remaining: 47.5s
650:	learn: 0.6718217	test: 0.6650586	best: 0.6650625 (649)	total: 1m 28s	remaining: 47.3s
651:	learn: 0.6718272	test: 0.6650495	best: 0.6650625 (649)	total: 1m 28s	remaining: 47.2s
652:	learn: 0.6718556	test: 0.6650648	best: 0.6650648 (652)	total: 1m 28s	remaining: 47.1s
653:	learn: 0.6718820	test: 0.6651026	best: 0.6651026 (653)	total: 1m 28s	remaining: 46.9s
654:	learn: 0.6718866	test: 0.6651091	best: 0.6651091 (654)	total: 1m 28s	remaining: 46.8s
655:	learn: 0.6719180	test: 0.6651089	best: 0.6651091 (654)	total: 1m 28s	remaining: 46.7s
656:	learn: 0.6719185	test: 0.6651011	best: 0.6651091 (654)	total: 1m 29s	remaining: 46.5s
657:	learn: 0.6719226	test: 0.6650685	best: 0.6651091 (654)	total: 1m 29s	remaining: 46.4s

739:	learn: 0.6729805	test: 0.6654080	best: 0.6654412 (733)	total: 1m 40s	remaining: 35.3s
740:	learn: 0.6729754	test: 0.6653366	best: 0.6654412 (733)	total: 1m 40s	remaining: 35.1s
741:	learn: 0.6730022	test: 0.6653273	best: 0.6654412 (733)	total: 1m 40s	remaining: 35s
742:	learn: 0.6730162	test: 0.6653806	best: 0.6654412 (733)	total: 1m 40s	remaining: 34.9s
743:	learn: 0.6730469	test: 0.6654083	best: 0.6654412 (733)	total: 1m 40s	remaining: 34.7s
744:	learn: 0.6730927	test: 0.6653972	best: 0.6654412 (733)	total: 1m 41s	remaining: 34.6s
745:	learn: 0.6730617	test: 0.6654301	best: 0.6654412 (733)	total: 1m 41s	remaining: 34.5s
746:	learn: 0.6730809	test: 0.6654176	best: 0.6654412 (733)	total: 1m 41s	remaining: 34.3s
747:	learn: 0.6730977	test: 0.6654248	best: 0.6654412 (733)	total: 1m 41s	remaining: 34.2s
748:	learn: 0.6731249	test: 0.6654207	best: 0.6654412 (733)	total: 1m 41s	remaining: 34.1s
749:	learn: 0.6731296	test: 0.6654139	best: 0.6654412 (733)	total: 1m 41s	remaining: 33.9s
7

831:	learn: 0.6738765	test: 0.6656397	best: 0.6656626 (830)	total: 1m 52s	remaining: 22.7s
832:	learn: 0.6739044	test: 0.6656497	best: 0.6656626 (830)	total: 1m 52s	remaining: 22.6s
833:	learn: 0.6739262	test: 0.6656544	best: 0.6656626 (830)	total: 1m 52s	remaining: 22.5s
834:	learn: 0.6739238	test: 0.6656687	best: 0.6656687 (834)	total: 1m 53s	remaining: 22.3s
835:	learn: 0.6739127	test: 0.6656888	best: 0.6656888 (835)	total: 1m 53s	remaining: 22.2s
836:	learn: 0.6738990	test: 0.6656950	best: 0.6656950 (836)	total: 1m 53s	remaining: 22.1s
837:	learn: 0.6739141	test: 0.6656583	best: 0.6656950 (836)	total: 1m 53s	remaining: 21.9s
838:	learn: 0.6739750	test: 0.6656774	best: 0.6656950 (836)	total: 1m 53s	remaining: 21.8s
839:	learn: 0.6739409	test: 0.6655999	best: 0.6656950 (836)	total: 1m 53s	remaining: 21.7s
840:	learn: 0.6739378	test: 0.6655598	best: 0.6656950 (836)	total: 1m 53s	remaining: 21.5s
841:	learn: 0.6739701	test: 0.6655926	best: 0.6656950 (836)	total: 1m 53s	remaining: 21.4s

923:	learn: 0.6750669	test: 0.6661142	best: 0.6661142 (923)	total: 2m 5s	remaining: 10.3s
924:	learn: 0.6750838	test: 0.6660954	best: 0.6661142 (923)	total: 2m 5s	remaining: 10.1s
925:	learn: 0.6750971	test: 0.6661282	best: 0.6661282 (925)	total: 2m 5s	remaining: 10s
926:	learn: 0.6751059	test: 0.6661360	best: 0.6661360 (926)	total: 2m 5s	remaining: 9.87s
927:	learn: 0.6750910	test: 0.6661142	best: 0.6661360 (926)	total: 2m 5s	remaining: 9.74s
928:	learn: 0.6751185	test: 0.6661093	best: 0.6661360 (926)	total: 2m 5s	remaining: 9.6s
929:	learn: 0.6751249	test: 0.6661208	best: 0.6661360 (926)	total: 2m 5s	remaining: 9.47s
930:	learn: 0.6751410	test: 0.6661019	best: 0.6661360 (926)	total: 2m 5s	remaining: 9.33s
931:	learn: 0.6751312	test: 0.6661414	best: 0.6661414 (931)	total: 2m 6s	remaining: 9.2s
932:	learn: 0.6751545	test: 0.6661534	best: 0.6661534 (932)	total: 2m 6s	remaining: 9.06s
933:	learn: 0.6751812	test: 0.6662270	best: 0.6662270 (933)	total: 2m 6s	remaining: 8.93s
934:	learn: 0.

In [114]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Recall', scale_pos_weight = 4.874)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.6823108	test: 0.6975478	best: 0.6975478 (0)	total: 164ms	remaining: 2m 43s
1:	learn: 0.7362257	test: 0.7433578	best: 0.7433578 (1)	total: 328ms	remaining: 2m 43s
2:	learn: 0.7364313	test: 0.7435343	best: 0.7435343 (2)	total: 477ms	remaining: 2m 38s
3:	learn: 0.7484875	test: 0.7509556	best: 0.7509556 (3)	total: 646ms	remaining: 2m 40s
4:	learn: 0.7579298	test: 0.7621994	best: 0.7621994 (4)	total: 781ms	remaining: 2m 35s
5:	learn: 0.7567770	test: 0.7610585	best: 0.7621994 (4)	total: 914ms	remaining: 2m 31s
6:	learn: 0.7562777	test: 0.7600823	best: 0.7621994 (4)	total: 1.09s	remaining: 2m 34s
7:	learn: 0.7525405	test: 0.7562364	best: 0.7621994 (4)	total: 1.26s	remaining: 2m 36s
8:	learn: 0.7531132	test: 0.7566010	best: 0.7621994 (4)	total: 1.42s	remaining: 2m 36s
9:	learn: 0.7551103	test: 0.7574831	best: 0.7621994 (4)	total: 1.56s	remaining: 2m 34s
10:	learn: 0.7554847	test: 0.7584593	best: 0.7621994 (4)	total: 1.69s	remaining: 2m 31s
11:	learn: 0

94:	learn: 0.7589944	test: 0.7636930	best: 0.7648221 (56)	total: 12.8s	remaining: 2m 1s
95:	learn: 0.7590678	test: 0.7636930	best: 0.7648221 (56)	total: 12.9s	remaining: 2m 1s
96:	learn: 0.7593175	test: 0.7640341	best: 0.7648221 (56)	total: 13.1s	remaining: 2m 1s
97:	learn: 0.7596772	test: 0.7643869	best: 0.7648221 (56)	total: 13.2s	remaining: 2m 1s
98:	learn: 0.7597139	test: 0.7641988	best: 0.7648221 (56)	total: 13.4s	remaining: 2m 1s
99:	learn: 0.7600297	test: 0.7647515	best: 0.7648221 (56)	total: 13.5s	remaining: 2m 1s
100:	learn: 0.7597433	test: 0.7643399	best: 0.7648221 (56)	total: 13.6s	remaining: 2m 1s
101:	learn: 0.7600150	test: 0.7643517	best: 0.7648221 (56)	total: 13.8s	remaining: 2m 1s
102:	learn: 0.7601325	test: 0.7643752	best: 0.7648221 (56)	total: 13.9s	remaining: 2m 1s
103:	learn: 0.7601398	test: 0.7644105	best: 0.7648221 (56)	total: 14.1s	remaining: 2m 1s
104:	learn: 0.7602059	test: 0.7645751	best: 0.7648221 (56)	total: 14.2s	remaining: 2m
105:	learn: 0.7603968	test: 0.

185:	learn: 0.7611163	test: 0.7625640	best: 0.7649515 (108)	total: 25.1s	remaining: 1m 49s
186:	learn: 0.7612705	test: 0.7623875	best: 0.7649515 (108)	total: 25.2s	remaining: 1m 49s
187:	learn: 0.7614394	test: 0.7625287	best: 0.7649515 (108)	total: 25.4s	remaining: 1m 49s
188:	learn: 0.7615275	test: 0.7627169	best: 0.7649515 (108)	total: 25.5s	remaining: 1m 49s
189:	learn: 0.7611237	test: 0.7627639	best: 0.7649515 (108)	total: 25.6s	remaining: 1m 49s
190:	learn: 0.7611016	test: 0.7627757	best: 0.7649515 (108)	total: 25.7s	remaining: 1m 48s
191:	learn: 0.7612265	test: 0.7627286	best: 0.7649515 (108)	total: 25.8s	remaining: 1m 48s
192:	learn: 0.7612265	test: 0.7628345	best: 0.7649515 (108)	total: 26s	remaining: 1m 48s
193:	learn: 0.7611971	test: 0.7626580	best: 0.7649515 (108)	total: 26.1s	remaining: 1m 48s
194:	learn: 0.7612705	test: 0.7625992	best: 0.7649515 (108)	total: 26.3s	remaining: 1m 48s
195:	learn: 0.7612852	test: 0.7624346	best: 0.7649515 (108)	total: 26.4s	remaining: 1m 48s
1

277:	learn: 0.7634732	test: 0.7632696	best: 0.7649515 (108)	total: 37s	remaining: 1m 36s
278:	learn: 0.7634806	test: 0.7633049	best: 0.7649515 (108)	total: 37.1s	remaining: 1m 35s
279:	learn: 0.7634292	test: 0.7632696	best: 0.7649515 (108)	total: 37.2s	remaining: 1m 35s
280:	learn: 0.7635173	test: 0.7633049	best: 0.7649515 (108)	total: 37.4s	remaining: 1m 35s
281:	learn: 0.7635687	test: 0.7633284	best: 0.7649515 (108)	total: 37.5s	remaining: 1m 35s
282:	learn: 0.7635613	test: 0.7633637	best: 0.7649515 (108)	total: 37.6s	remaining: 1m 35s
283:	learn: 0.7635834	test: 0.7634108	best: 0.7649515 (108)	total: 37.8s	remaining: 1m 35s
284:	learn: 0.7636494	test: 0.7633990	best: 0.7649515 (108)	total: 37.9s	remaining: 1m 35s
285:	learn: 0.7636494	test: 0.7633872	best: 0.7649515 (108)	total: 38s	remaining: 1m 34s
286:	learn: 0.7636568	test: 0.7631755	best: 0.7649515 (108)	total: 38.2s	remaining: 1m 34s
287:	learn: 0.7634659	test: 0.7631520	best: 0.7649515 (108)	total: 38.3s	remaining: 1m 34s
288

369:	learn: 0.7648536	test: 0.7627874	best: 0.7649515 (108)	total: 49.4s	remaining: 1m 24s
370:	learn: 0.7646186	test: 0.7626463	best: 0.7649515 (108)	total: 49.5s	remaining: 1m 23s
371:	learn: 0.7647508	test: 0.7626698	best: 0.7649515 (108)	total: 49.7s	remaining: 1m 23s
372:	learn: 0.7647728	test: 0.7627404	best: 0.7649515 (108)	total: 49.8s	remaining: 1m 23s
373:	learn: 0.7648242	test: 0.7627757	best: 0.7649515 (108)	total: 49.9s	remaining: 1m 23s
374:	learn: 0.7648389	test: 0.7627639	best: 0.7649515 (108)	total: 50s	remaining: 1m 23s
375:	learn: 0.7648536	test: 0.7627639	best: 0.7649515 (108)	total: 50.2s	remaining: 1m 23s
376:	learn: 0.7649197	test: 0.7628580	best: 0.7649515 (108)	total: 50.3s	remaining: 1m 23s
377:	learn: 0.7649564	test: 0.7628462	best: 0.7649515 (108)	total: 50.4s	remaining: 1m 23s
378:	learn: 0.7649270	test: 0.7628580	best: 0.7649515 (108)	total: 50.6s	remaining: 1m 22s
379:	learn: 0.7649784	test: 0.7627521	best: 0.7649515 (108)	total: 50.7s	remaining: 1m 22s
3

...

552:	learn: 0.7679594	test: 0.7619171	best: 0.7649515 (108)	total: 1m 14s	remaining: 59.9s
553:	learn: 0.7679961	test: 0.7617877	best: 0.7649515 (108)	total: 1m 14s	remaining: 59.7s
554:	learn: 0.7680255	test: 0.7617524	best: 0.7649515 (108)	total: 1m 14s	remaining: 59.6s
555:	learn: 0.7680402	test: 0.7616936	best: 0.7649515 (108)	total: 1m 14s	remaining: 59.5s
556:	learn: 0.7681356	test: 0.7618700	best: 0.7649515 (108)	total: 1m 14s	remaining: 59.3s
557:	learn: 0.7681650	test: 0.7618230	best: 0.7649515 (108)	total: 1m 14s	remaining: 59.2s
558:	learn: 0.7682017	test: 0.7618348	best: 0.7649515 (108)	total: 1m 14s	remaining: 59.1s
559:	learn: 0.7682091	test: 0.7618465	best: 0.7649515 (108)	total: 1m 14s	remaining: 58.9s
560:	learn: 0.7681577	test: 0.7618818	best: 0.7649515 (108)	total: 1m 15s	remaining: 58.8s
561:	learn: 0.7681577	test: 0.7618818	best: 0.7649515 (108)	total: 1m 15s	remaining: 58.7s
562:	learn: 0.7682017	test: 0.7618818	best: 0.7649515 (108)	total: 1m 15s	remaining: 58.5s

644:	learn: 0.7695454	test: 0.7619994	best: 0.7649515 (108)	total: 1m 26s	remaining: 47.8s
645:	learn: 0.7694793	test: 0.7618465	best: 0.7649515 (108)	total: 1m 27s	remaining: 47.7s
646:	learn: 0.7694205	test: 0.7618230	best: 0.7649515 (108)	total: 1m 27s	remaining: 47.5s
647:	learn: 0.7694279	test: 0.7618465	best: 0.7649515 (108)	total: 1m 27s	remaining: 47.4s
648:	learn: 0.7694352	test: 0.7618348	best: 0.7649515 (108)	total: 1m 27s	remaining: 47.3s
649:	learn: 0.7694352	test: 0.7618112	best: 0.7649515 (108)	total: 1m 27s	remaining: 47.1s
650:	learn: 0.7695674	test: 0.7618230	best: 0.7649515 (108)	total: 1m 27s	remaining: 47s
651:	learn: 0.7694719	test: 0.7618112	best: 0.7649515 (108)	total: 1m 27s	remaining: 46.9s
652:	learn: 0.7695894	test: 0.7617995	best: 0.7649515 (108)	total: 1m 27s	remaining: 46.7s
653:	learn: 0.7695454	test: 0.7617642	best: 0.7649515 (108)	total: 1m 28s	remaining: 46.6s
654:	learn: 0.7695747	test: 0.7618348	best: 0.7649515 (108)	total: 1m 28s	remaining: 46.5s
6

736:	learn: 0.7707715	test: 0.7613996	best: 0.7649515 (108)	total: 1m 39s	remaining: 35.4s
737:	learn: 0.7707862	test: 0.7615172	best: 0.7649515 (108)	total: 1m 39s	remaining: 35.3s
738:	learn: 0.7707642	test: 0.7614819	best: 0.7649515 (108)	total: 1m 39s	remaining: 35.2s
739:	learn: 0.7707422	test: 0.7614466	best: 0.7649515 (108)	total: 1m 39s	remaining: 35s
740:	learn: 0.7707789	test: 0.7614937	best: 0.7649515 (108)	total: 1m 39s	remaining: 34.9s
741:	learn: 0.7708376	test: 0.7615172	best: 0.7649515 (108)	total: 1m 39s	remaining: 34.7s
742:	learn: 0.7708596	test: 0.7615054	best: 0.7649515 (108)	total: 1m 40s	remaining: 34.6s
743:	learn: 0.7708523	test: 0.7614702	best: 0.7649515 (108)	total: 1m 40s	remaining: 34.5s
744:	learn: 0.7708743	test: 0.7614702	best: 0.7649515 (108)	total: 1m 40s	remaining: 34.3s
745:	learn: 0.7709551	test: 0.7613643	best: 0.7649515 (108)	total: 1m 40s	remaining: 34.2s
746:	learn: 0.7708376	test: 0.7612937	best: 0.7649515 (108)	total: 1m 40s	remaining: 34.1s
7

828:	learn: 0.7713736	test: 0.7601764	best: 0.7649515 (108)	total: 1m 51s	remaining: 23s
829:	learn: 0.7713369	test: 0.7601882	best: 0.7649515 (108)	total: 1m 51s	remaining: 22.8s
830:	learn: 0.7713516	test: 0.7600118	best: 0.7649515 (108)	total: 1m 51s	remaining: 22.7s
831:	learn: 0.7714691	test: 0.7598589	best: 0.7649515 (108)	total: 1m 51s	remaining: 22.6s
832:	learn: 0.7713075	test: 0.7600118	best: 0.7649515 (108)	total: 1m 51s	remaining: 22.4s
833:	learn: 0.7713369	test: 0.7602117	best: 0.7649515 (108)	total: 1m 51s	remaining: 22.3s
834:	learn: 0.7713222	test: 0.7600706	best: 0.7649515 (108)	total: 1m 52s	remaining: 22.1s
835:	learn: 0.7713149	test: 0.7600823	best: 0.7649515 (108)	total: 1m 52s	remaining: 22s
836:	learn: 0.7712782	test: 0.7600353	best: 0.7649515 (108)	total: 1m 52s	remaining: 21.9s
837:	learn: 0.7713736	test: 0.7600235	best: 0.7649515 (108)	total: 1m 52s	remaining: 21.7s
838:	learn: 0.7713810	test: 0.7598706	best: 0.7649515 (108)	total: 1m 52s	remaining: 21.6s
839

919:	learn: 0.7723501	test: 0.7590473	best: 0.7649515 (108)	total: 2m 3s	remaining: 10.8s
920:	learn: 0.7723575	test: 0.7591179	best: 0.7649515 (108)	total: 2m 3s	remaining: 10.6s
921:	learn: 0.7724015	test: 0.7590944	best: 0.7649515 (108)	total: 2m 3s	remaining: 10.5s
922:	learn: 0.7724089	test: 0.7591767	best: 0.7649515 (108)	total: 2m 4s	remaining: 10.4s
923:	learn: 0.7723355	test: 0.7591061	best: 0.7649515 (108)	total: 2m 4s	remaining: 10.2s
924:	learn: 0.7723501	test: 0.7588356	best: 0.7649515 (108)	total: 2m 4s	remaining: 10.1s
925:	learn: 0.7723501	test: 0.7589180	best: 0.7649515 (108)	total: 2m 4s	remaining: 9.95s
926:	learn: 0.7723134	test: 0.7589650	best: 0.7649515 (108)	total: 2m 4s	remaining: 9.81s
927:	learn: 0.7722547	test: 0.7590003	best: 0.7649515 (108)	total: 2m 4s	remaining: 9.68s
928:	learn: 0.7722400	test: 0.7591297	best: 0.7649515 (108)	total: 2m 4s	remaining: 9.54s
929:	learn: 0.7723575	test: 0.7590826	best: 0.7649515 (108)	total: 2m 5s	remaining: 9.41s
930:	learn

In [116]:
catpred2 = clf.predict(X2)
np.savetxt("catpred2.csv", catpred2, delimiter=",")

In [117]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Logloss', scale_pos_weight = 0.2)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.4748641	test: 0.4746349	best: 0.4746349 (0)	total: 134ms	remaining: 2m 13s
1:	learn: 0.3475862	test: 0.3470871	best: 0.3470871 (1)	total: 267ms	remaining: 2m 13s
2:	learn: 0.2727768	test: 0.2711609	best: 0.2711609 (2)	total: 398ms	remaining: 2m 12s
3:	learn: 0.2285679	test: 0.2274491	best: 0.2274491 (3)	total: 540ms	remaining: 2m 14s
4:	learn: 0.2017283	test: 0.2007512	best: 0.2007512 (4)	total: 684ms	remaining: 2m 16s
5:	learn: 0.1849215	test: 0.1840994	best: 0.1840994 (5)	total: 811ms	remaining: 2m 14s
6:	learn: 0.1740727	test: 0.1733654	best: 0.1733654 (6)	total: 968ms	remaining: 2m 17s
7:	learn: 0.1671158	test: 0.1664591	best: 0.1664591 (7)	total: 1.11s	remaining: 2m 18s
8:	learn: 0.1622670	test: 0.1616397	best: 0.1616397 (8)	total: 1.27s	remaining: 2m 20s
9:	learn: 0.1592239	test: 0.1585459	best: 0.1585459 (9)	total: 1.41s	remaining: 2m 19s
10:	learn: 0.1570769	test: 0.1564625	best: 0.1564625 (10)	total: 1.57s	remaining: 2m 21s
11:	learn: 

94:	learn: 0.1503247	test: 0.1501825	best: 0.1501825 (94)	total: 12.8s	remaining: 2m 1s
95:	learn: 0.1503199	test: 0.1501805	best: 0.1501805 (95)	total: 12.9s	remaining: 2m 1s
96:	learn: 0.1503143	test: 0.1501763	best: 0.1501763 (96)	total: 13.1s	remaining: 2m 1s
97:	learn: 0.1503064	test: 0.1501696	best: 0.1501696 (97)	total: 13.2s	remaining: 2m 1s
98:	learn: 0.1503009	test: 0.1501677	best: 0.1501677 (98)	total: 13.3s	remaining: 2m 1s
99:	learn: 0.1502965	test: 0.1501691	best: 0.1501677 (98)	total: 13.4s	remaining: 2m
100:	learn: 0.1502913	test: 0.1501636	best: 0.1501636 (100)	total: 13.6s	remaining: 2m
101:	learn: 0.1502866	test: 0.1501614	best: 0.1501614 (101)	total: 13.7s	remaining: 2m
102:	learn: 0.1502857	test: 0.1501612	best: 0.1501612 (102)	total: 13.8s	remaining: 2m
103:	learn: 0.1502814	test: 0.1501582	best: 0.1501582 (103)	total: 14s	remaining: 2m
104:	learn: 0.1502730	test: 0.1501483	best: 0.1501483 (104)	total: 14.1s	remaining: 2m
105:	learn: 0.1502664	test: 0.1501426	best

186:	learn: 0.1498343	test: 0.1499227	best: 0.1499223 (185)	total: 24.8s	remaining: 1m 48s
187:	learn: 0.1498267	test: 0.1499177	best: 0.1499177 (187)	total: 25s	remaining: 1m 47s
188:	learn: 0.1498206	test: 0.1499153	best: 0.1499153 (188)	total: 25.1s	remaining: 1m 47s
189:	learn: 0.1498197	test: 0.1499153	best: 0.1499153 (188)	total: 25.2s	remaining: 1m 47s
190:	learn: 0.1498120	test: 0.1499092	best: 0.1499092 (190)	total: 25.3s	remaining: 1m 47s
191:	learn: 0.1498012	test: 0.1498979	best: 0.1498979 (191)	total: 25.5s	remaining: 1m 47s
192:	learn: 0.1497992	test: 0.1498985	best: 0.1498979 (191)	total: 25.6s	remaining: 1m 47s
193:	learn: 0.1497975	test: 0.1498972	best: 0.1498972 (193)	total: 25.7s	remaining: 1m 46s
194:	learn: 0.1497957	test: 0.1498964	best: 0.1498964 (194)	total: 25.8s	remaining: 1m 46s
195:	learn: 0.1497920	test: 0.1498948	best: 0.1498948 (195)	total: 25.9s	remaining: 1m 46s
196:	learn: 0.1497808	test: 0.1498831	best: 0.1498831 (196)	total: 26.1s	remaining: 1m 46s
1

278:	learn: 0.1494834	test: 0.1497209	best: 0.1497209 (278)	total: 36.8s	remaining: 1m 35s
279:	learn: 0.1494801	test: 0.1497195	best: 0.1497195 (279)	total: 36.9s	remaining: 1m 34s
280:	learn: 0.1494774	test: 0.1497195	best: 0.1497195 (279)	total: 37.1s	remaining: 1m 34s
281:	learn: 0.1494759	test: 0.1497194	best: 0.1497194 (281)	total: 37.2s	remaining: 1m 34s
282:	learn: 0.1494746	test: 0.1497183	best: 0.1497183 (282)	total: 37.4s	remaining: 1m 34s
283:	learn: 0.1494715	test: 0.1497193	best: 0.1497183 (282)	total: 37.5s	remaining: 1m 34s
284:	learn: 0.1494673	test: 0.1497167	best: 0.1497167 (284)	total: 37.6s	remaining: 1m 34s
285:	learn: 0.1494612	test: 0.1497145	best: 0.1497145 (285)	total: 37.8s	remaining: 1m 34s
286:	learn: 0.1494586	test: 0.1497137	best: 0.1497137 (286)	total: 37.9s	remaining: 1m 34s
287:	learn: 0.1494562	test: 0.1497141	best: 0.1497137 (286)	total: 38s	remaining: 1m 33s
288:	learn: 0.1494543	test: 0.1497120	best: 0.1497120 (288)	total: 38.1s	remaining: 1m 33s
2

370:	learn: 0.1492401	test: 0.1496599	best: 0.1496597 (363)	total: 48.7s	remaining: 1m 22s
371:	learn: 0.1492391	test: 0.1496598	best: 0.1496597 (363)	total: 48.8s	remaining: 1m 22s
372:	learn: 0.1492361	test: 0.1496593	best: 0.1496593 (372)	total: 49s	remaining: 1m 22s
373:	learn: 0.1492337	test: 0.1496595	best: 0.1496593 (372)	total: 49.1s	remaining: 1m 22s
374:	learn: 0.1492328	test: 0.1496601	best: 0.1496593 (372)	total: 49.2s	remaining: 1m 22s
375:	learn: 0.1492284	test: 0.1496579	best: 0.1496579 (375)	total: 49.4s	remaining: 1m 21s
376:	learn: 0.1492230	test: 0.1496539	best: 0.1496539 (376)	total: 49.5s	remaining: 1m 21s
377:	learn: 0.1492221	test: 0.1496535	best: 0.1496535 (377)	total: 49.6s	remaining: 1m 21s
378:	learn: 0.1492215	test: 0.1496539	best: 0.1496535 (377)	total: 49.7s	remaining: 1m 21s
379:	learn: 0.1492194	test: 0.1496519	best: 0.1496519 (379)	total: 49.8s	remaining: 1m 21s
380:	learn: 0.1492171	test: 0.1496523	best: 0.1496519 (379)	total: 50s	remaining: 1m 21s
381

...

554:	learn: 0.1488215	test: 0.1495638	best: 0.1495627 (553)	total: 1m 13s	remaining: 58.7s
555:	learn: 0.1488206	test: 0.1495640	best: 0.1495627 (553)	total: 1m 13s	remaining: 58.6s
556:	learn: 0.1488184	test: 0.1495646	best: 0.1495627 (553)	total: 1m 13s	remaining: 58.4s
557:	learn: 0.1488154	test: 0.1495614	best: 0.1495614 (557)	total: 1m 13s	remaining: 58.3s
558:	learn: 0.1488146	test: 0.1495616	best: 0.1495614 (557)	total: 1m 13s	remaining: 58.2s
559:	learn: 0.1488126	test: 0.1495629	best: 0.1495614 (557)	total: 1m 13s	remaining: 58s
560:	learn: 0.1488101	test: 0.1495622	best: 0.1495614 (557)	total: 1m 14s	remaining: 57.9s
561:	learn: 0.1488090	test: 0.1495620	best: 0.1495614 (557)	total: 1m 14s	remaining: 57.8s
562:	learn: 0.1488084	test: 0.1495624	best: 0.1495614 (557)	total: 1m 14s	remaining: 57.6s
563:	learn: 0.1488077	test: 0.1495629	best: 0.1495614 (557)	total: 1m 14s	remaining: 57.5s
564:	learn: 0.1488076	test: 0.1495627	best: 0.1495614 (557)	total: 1m 14s	remaining: 57.4s
5

646:	learn: 0.1486622	test: 0.1495451	best: 0.1495446 (622)	total: 1m 25s	remaining: 46.4s
647:	learn: 0.1486600	test: 0.1495459	best: 0.1495446 (622)	total: 1m 25s	remaining: 46.2s
648:	learn: 0.1486569	test: 0.1495462	best: 0.1495446 (622)	total: 1m 25s	remaining: 46.1s
649:	learn: 0.1486550	test: 0.1495454	best: 0.1495446 (622)	total: 1m 25s	remaining: 46s
650:	learn: 0.1486543	test: 0.1495457	best: 0.1495446 (622)	total: 1m 25s	remaining: 45.8s
651:	learn: 0.1486537	test: 0.1495460	best: 0.1495446 (622)	total: 1m 25s	remaining: 45.7s
652:	learn: 0.1486522	test: 0.1495448	best: 0.1495446 (622)	total: 1m 25s	remaining: 45.6s
653:	learn: 0.1486452	test: 0.1495397	best: 0.1495397 (653)	total: 1m 25s	remaining: 45.4s
654:	learn: 0.1486448	test: 0.1495400	best: 0.1495397 (653)	total: 1m 25s	remaining: 45.3s
655:	learn: 0.1486420	test: 0.1495392	best: 0.1495392 (655)	total: 1m 26s	remaining: 45.2s
656:	learn: 0.1486405	test: 0.1495395	best: 0.1495392 (655)	total: 1m 26s	remaining: 45s
657

738:	learn: 0.1485110	test: 0.1495361	best: 0.1495330 (735)	total: 1m 37s	remaining: 34.3s
739:	learn: 0.1485098	test: 0.1495364	best: 0.1495330 (735)	total: 1m 37s	remaining: 34.1s
740:	learn: 0.1485088	test: 0.1495370	best: 0.1495330 (735)	total: 1m 37s	remaining: 34s
741:	learn: 0.1485070	test: 0.1495371	best: 0.1495330 (735)	total: 1m 37s	remaining: 33.9s
742:	learn: 0.1485057	test: 0.1495373	best: 0.1495330 (735)	total: 1m 37s	remaining: 33.8s
743:	learn: 0.1485032	test: 0.1495372	best: 0.1495330 (735)	total: 1m 37s	remaining: 33.6s
744:	learn: 0.1485017	test: 0.1495371	best: 0.1495330 (735)	total: 1m 37s	remaining: 33.5s
745:	learn: 0.1484938	test: 0.1495307	best: 0.1495307 (745)	total: 1m 37s	remaining: 33.4s
746:	learn: 0.1484929	test: 0.1495311	best: 0.1495307 (745)	total: 1m 38s	remaining: 33.2s
747:	learn: 0.1484913	test: 0.1495304	best: 0.1495304 (747)	total: 1m 38s	remaining: 33.1s
748:	learn: 0.1484897	test: 0.1495306	best: 0.1495304 (747)	total: 1m 38s	remaining: 33s
749

830:	learn: 0.1483284	test: 0.1495038	best: 0.1495037 (828)	total: 1m 48s	remaining: 22.1s
831:	learn: 0.1483278	test: 0.1495036	best: 0.1495036 (831)	total: 1m 48s	remaining: 22s
832:	learn: 0.1483271	test: 0.1495032	best: 0.1495032 (832)	total: 1m 49s	remaining: 21.9s
833:	learn: 0.1483244	test: 0.1495028	best: 0.1495028 (833)	total: 1m 49s	remaining: 21.7s
834:	learn: 0.1483236	test: 0.1495025	best: 0.1495025 (834)	total: 1m 49s	remaining: 21.6s
835:	learn: 0.1483225	test: 0.1495025	best: 0.1495025 (835)	total: 1m 49s	remaining: 21.5s
836:	learn: 0.1483197	test: 0.1494998	best: 0.1494998 (836)	total: 1m 49s	remaining: 21.3s
837:	learn: 0.1483167	test: 0.1494987	best: 0.1494987 (837)	total: 1m 49s	remaining: 21.2s
838:	learn: 0.1483150	test: 0.1494981	best: 0.1494981 (838)	total: 1m 49s	remaining: 21.1s
839:	learn: 0.1483143	test: 0.1494973	best: 0.1494973 (839)	total: 1m 49s	remaining: 20.9s
840:	learn: 0.1483120	test: 0.1494963	best: 0.1494963 (840)	total: 1m 50s	remaining: 20.8s
8

922:	learn: 0.1481771	test: 0.1494771	best: 0.1494771 (922)	total: 2m	remaining: 10s
923:	learn: 0.1481749	test: 0.1494762	best: 0.1494762 (923)	total: 2m	remaining: 9.9s
924:	learn: 0.1481741	test: 0.1494761	best: 0.1494761 (924)	total: 2m	remaining: 9.78s
925:	learn: 0.1481718	test: 0.1494763	best: 0.1494761 (924)	total: 2m	remaining: 9.65s
926:	learn: 0.1481695	test: 0.1494781	best: 0.1494761 (924)	total: 2m	remaining: 9.52s
927:	learn: 0.1481673	test: 0.1494769	best: 0.1494761 (924)	total: 2m	remaining: 9.38s
928:	learn: 0.1481662	test: 0.1494777	best: 0.1494761 (924)	total: 2m 1s	remaining: 9.25s
929:	learn: 0.1481644	test: 0.1494783	best: 0.1494761 (924)	total: 2m 1s	remaining: 9.12s
930:	learn: 0.1481631	test: 0.1494790	best: 0.1494761 (924)	total: 2m 1s	remaining: 8.99s
931:	learn: 0.1481599	test: 0.1494778	best: 0.1494761 (924)	total: 2m 1s	remaining: 8.86s
932:	learn: 0.1481598	test: 0.1494777	best: 0.1494761 (924)	total: 2m 1s	remaining: 8.73s
933:	learn: 0.1481573	test: 0.1

In [87]:
len(catpred)

500000

In [88]:
len(y2)

500000

In [89]:
np.savetxt("catpred.csv", catpred, delimiter=",")

In [90]:
X2

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,...,C16,C17,C18,C19,C20,C21,day_of_year,weekday,week_of_year,day_of_month
0,1005,0,85f751fd,c4e18dd6,50e219e0,2bc6bb64,7801e8d9,0f2161f8,a99f214a,763db4a4,...,50,2036,3,47,-1,156,294,1,43,21
1,1012,0,60e4b83b,4c984583,50e219e0,ecad2386,7801e8d9,07d7df22,a99f214a,18b355bf,...,50,2647,2,35,100148,23,301,1,44,28
2,1005,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,0f2161f8,a99f214a,f92b8696,...,50,2480,3,297,100111,61,299,6,43,26
3,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,0f2161f8,0ab2885f,0fc3ba9b,...,50,2374,3,39,-1,23,296,3,43,23
4,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,b56c0efd,...,50,2664,0,35,100083,51,302,2,44,29
5,1005,0,85f751fd,c4e18dd6,50e219e0,3e2bf98d,2347f47a,0f2161f8,54fdca59,9e4348d4,...,50,2506,0,35,-1,157,295,2,43,22
6,1005,0,85f751fd,c4e18dd6,50e219e0,a0fc55e5,2347f47a,0f2161f8,995c37f7,5e27d4c8,...,50,2476,2,167,100013,23,296,3,43,23
7,1005,0,85f751fd,c4e18dd6,50e219e0,a4869716,2347f47a,d1327cf5,a99f214a,dcd43cfa,...,50,112,3,1319,-1,15,297,4,43,24
8,1005,0,85f751fd,c4e18dd6,50e219e0,396df801,2347f47a,0f2161f8,a99f214a,a3eb1643,...,50,2487,1,547,-1,51,294,1,43,21
9,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,0f2161f8,2e3a3f8f,b5791b33,...,50,423,2,39,-1,32,297,4,43,24


In [26]:
def Snippet_200(): 
    print()
    print(format('How to find optimal parameters for CatBoost using GridSearchCV for Classification','*^82'))    
    
    import warnings
    warnings.filterwarnings("ignore")
    
    # load libraries
    from sklearn import datasets
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import RandomizedSearchCV
    from scipy.stats import uniform as sp_randFloat
    from scipy.stats import randint as sp_randInt    
    from catboost import CatBoostClassifier

    # load the iris datasets

    model = CatBoostClassifier()
    parameters = {'depth'         : sp_randInt(4, 10),
                  'learning_rate' : sp_randFloat(0.03, 0.11),
                  'iterations'    : sp_randInt(10, 100),
                  'l2_leaf_reg'    : sp_randInt(1, 9),
                  'scale_pos_weight'    : sp_randFloat(3, 6),
                  
                 }
    
    randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 2, n_iter = 10, n_jobs=-1)
    randm.fit(X_train, y_train, cat_features = categorical_feature_indices)

    # Results from Random Search
    print("\n========================================================")
    print(" Results from Random Search " )
    print("========================================================")    
    
    print("\n The best estimator across ALL searched params:\n",
          randm.best_estimator_)
    
    print("\n The best score across ALL searched params:\n",
          randm.best_score_)
    
    print("\n The best parameters across ALL searched params:\n",
          randm.best_params_)
    
    print("\n ========================================================")
Snippet_200()


How to find optimal parameters for CatBoost using GridSearchCV for Classification*
0:	learn: 0.6705794	total: 662ms	remaining: 55.6s
1:	learn: 0.6541593	total: 1.19s	remaining: 49.5s
2:	learn: 0.6422978	total: 1.78s	remaining: 48.6s
3:	learn: 0.6331775	total: 2.41s	remaining: 48.8s
4:	learn: 0.6265757	total: 2.98s	remaining: 47.7s
5:	learn: 0.6216045	total: 3.56s	remaining: 46.8s
6:	learn: 0.6179566	total: 4.13s	remaining: 46s
7:	learn: 0.6147219	total: 4.82s	remaining: 46.4s
8:	learn: 0.6122981	total: 5.45s	remaining: 46s
9:	learn: 0.6104108	total: 6.16s	remaining: 46.2s
10:	learn: 0.6086097	total: 6.73s	remaining: 45.3s
11:	learn: 0.6073892	total: 7.28s	remaining: 44.3s
12:	learn: 0.6060774	total: 7.87s	remaining: 43.6s
13:	learn: 0.6052068	total: 8.43s	remaining: 42.7s
14:	learn: 0.6044351	total: 8.98s	remaining: 41.9s
15:	learn: 0.6039483	total: 9.54s	remaining: 41.1s
16:	learn: 0.6032877	total: 10.1s	remaining: 40.5s
17:	learn: 0.6023247	total: 10.8s	remaining: 40.2s
18:	learn: 0

In [87]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Logloss', scale_pos_weight = 4.874)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())
print(clf.get_params())

Learning rate set to 0.157426
0:	learn: 0.6663183	test: 0.6655905	best: 0.6655905 (0)	total: 169ms	remaining: 2m 48s
1:	learn: 0.6480075	test: 0.6466006	best: 0.6466006 (1)	total: 348ms	remaining: 2m 53s
2:	learn: 0.6350584	test: 0.6332874	best: 0.6332874 (2)	total: 518ms	remaining: 2m 52s
3:	learn: 0.6258203	test: 0.6241241	best: 0.6241241 (3)	total: 670ms	remaining: 2m 46s
4:	learn: 0.6191749	test: 0.6173932	best: 0.6173932 (4)	total: 848ms	remaining: 2m 48s
5:	learn: 0.6144669	test: 0.6124856	best: 0.6124856 (5)	total: 1s	remaining: 2m 46s
6:	learn: 0.6108735	test: 0.6092502	best: 0.6092502 (6)	total: 1.17s	remaining: 2m 45s
7:	learn: 0.6079224	test: 0.6059914	best: 0.6059914 (7)	total: 1.31s	remaining: 2m 42s
8:	learn: 0.6057963	test: 0.6039926	best: 0.6039926 (8)	total: 1.46s	remaining: 2m 40s
9:	learn: 0.6043187	test: 0.6024545	best: 0.6024545 (9)	total: 1.61s	remaining: 2m 39s
10:	learn: 0.6029936	test: 0.6011312	best: 0.6011312 (10)	total: 1.79s	remaining: 2m 40s
11:	learn: 0.6

93:	learn: 0.5873603	test: 0.5870743	best: 0.5870743 (93)	total: 13.1s	remaining: 2m 5s
94:	learn: 0.5872900	test: 0.5869970	best: 0.5869970 (94)	total: 13.2s	remaining: 2m 5s
95:	learn: 0.5872284	test: 0.5869507	best: 0.5869507 (95)	total: 13.4s	remaining: 2m 5s
96:	learn: 0.5871483	test: 0.5868824	best: 0.5868824 (96)	total: 13.5s	remaining: 2m 5s
97:	learn: 0.5870729	test: 0.5868310	best: 0.5868310 (97)	total: 13.6s	remaining: 2m 5s
98:	learn: 0.5870300	test: 0.5867957	best: 0.5867957 (98)	total: 13.8s	remaining: 2m 5s
99:	learn: 0.5869222	test: 0.5866993	best: 0.5866993 (99)	total: 13.9s	remaining: 2m 5s
100:	learn: 0.5868695	test: 0.5866679	best: 0.5866679 (100)	total: 14.1s	remaining: 2m 5s
101:	learn: 0.5867935	test: 0.5866318	best: 0.5866318 (101)	total: 14.2s	remaining: 2m 4s
102:	learn: 0.5867528	test: 0.5866069	best: 0.5866069 (102)	total: 14.3s	remaining: 2m 4s
103:	learn: 0.5867182	test: 0.5865719	best: 0.5865719 (103)	total: 14.4s	remaining: 2m 4s
104:	learn: 0.5866627	te

185:	learn: 0.5829866	test: 0.5845199	best: 0.5845199 (185)	total: 25.4s	remaining: 1m 51s
186:	learn: 0.5829475	test: 0.5844941	best: 0.5844941 (186)	total: 25.5s	remaining: 1m 50s
187:	learn: 0.5829018	test: 0.5844593	best: 0.5844593 (187)	total: 25.7s	remaining: 1m 51s
188:	learn: 0.5828723	test: 0.5844568	best: 0.5844568 (188)	total: 25.8s	remaining: 1m 50s
189:	learn: 0.5828603	test: 0.5844587	best: 0.5844568 (188)	total: 25.9s	remaining: 1m 50s
190:	learn: 0.5828076	test: 0.5844183	best: 0.5844183 (190)	total: 26.1s	remaining: 1m 50s
191:	learn: 0.5827502	test: 0.5843848	best: 0.5843848 (191)	total: 26.2s	remaining: 1m 50s
192:	learn: 0.5827052	test: 0.5843610	best: 0.5843610 (192)	total: 26.4s	remaining: 1m 50s
193:	learn: 0.5826639	test: 0.5843365	best: 0.5843365 (193)	total: 26.5s	remaining: 1m 50s
194:	learn: 0.5826274	test: 0.5843200	best: 0.5843200 (194)	total: 26.6s	remaining: 1m 49s
195:	learn: 0.5826014	test: 0.5843064	best: 0.5843064 (195)	total: 26.8s	remaining: 1m 49s

277:	learn: 0.5802263	test: 0.5833797	best: 0.5833797 (277)	total: 38.2s	remaining: 1m 39s
278:	learn: 0.5801827	test: 0.5833813	best: 0.5833797 (277)	total: 38.3s	remaining: 1m 38s
279:	learn: 0.5801630	test: 0.5833816	best: 0.5833797 (277)	total: 38.4s	remaining: 1m 38s
280:	learn: 0.5801409	test: 0.5833808	best: 0.5833797 (277)	total: 38.5s	remaining: 1m 38s
281:	learn: 0.5801249	test: 0.5833837	best: 0.5833797 (277)	total: 38.6s	remaining: 1m 38s
282:	learn: 0.5801010	test: 0.5833699	best: 0.5833699 (282)	total: 38.8s	remaining: 1m 38s
283:	learn: 0.5800875	test: 0.5833694	best: 0.5833694 (283)	total: 38.9s	remaining: 1m 38s
284:	learn: 0.5800622	test: 0.5833728	best: 0.5833694 (283)	total: 39s	remaining: 1m 37s
285:	learn: 0.5800516	test: 0.5833701	best: 0.5833694 (283)	total: 39.2s	remaining: 1m 37s
286:	learn: 0.5800222	test: 0.5833514	best: 0.5833514 (286)	total: 39.3s	remaining: 1m 37s
287:	learn: 0.5799936	test: 0.5833416	best: 0.5833416 (287)	total: 39.5s	remaining: 1m 37s
2

369:	learn: 0.5779901	test: 0.5827885	best: 0.5827841 (366)	total: 50.8s	remaining: 1m 26s
370:	learn: 0.5779649	test: 0.5827794	best: 0.5827794 (370)	total: 51s	remaining: 1m 26s
371:	learn: 0.5779342	test: 0.5827777	best: 0.5827777 (371)	total: 51.1s	remaining: 1m 26s
372:	learn: 0.5779141	test: 0.5827705	best: 0.5827705 (372)	total: 51.2s	remaining: 1m 26s
373:	learn: 0.5778858	test: 0.5827699	best: 0.5827699 (373)	total: 51.3s	remaining: 1m 25s
374:	learn: 0.5778483	test: 0.5827434	best: 0.5827434 (374)	total: 51.5s	remaining: 1m 25s
375:	learn: 0.5778320	test: 0.5827473	best: 0.5827434 (374)	total: 51.6s	remaining: 1m 25s
376:	learn: 0.5778075	test: 0.5827515	best: 0.5827434 (374)	total: 51.8s	remaining: 1m 25s
377:	learn: 0.5777847	test: 0.5827494	best: 0.5827434 (374)	total: 51.9s	remaining: 1m 25s
378:	learn: 0.5777662	test: 0.5827435	best: 0.5827434 (374)	total: 52s	remaining: 1m 25s
379:	learn: 0.5777341	test: 0.5827394	best: 0.5827394 (379)	total: 52.1s	remaining: 1m 25s
380

...

552:	learn: 0.5741725	test: 0.5821750	best: 0.5821519 (494)	total: 1m 15s	remaining: 1m 1s
553:	learn: 0.5741561	test: 0.5821777	best: 0.5821519 (494)	total: 1m 15s	remaining: 1m
554:	learn: 0.5741389	test: 0.5821756	best: 0.5821519 (494)	total: 1m 15s	remaining: 1m
555:	learn: 0.5741233	test: 0.5821685	best: 0.5821519 (494)	total: 1m 15s	remaining: 1m
556:	learn: 0.5741096	test: 0.5821718	best: 0.5821519 (494)	total: 1m 16s	remaining: 1m
557:	learn: 0.5740837	test: 0.5821868	best: 0.5821519 (494)	total: 1m 16s	remaining: 1m
558:	learn: 0.5740640	test: 0.5821827	best: 0.5821519 (494)	total: 1m 16s	remaining: 1m
559:	learn: 0.5740567	test: 0.5821813	best: 0.5821519 (494)	total: 1m 16s	remaining: 1m
560:	learn: 0.5740395	test: 0.5821801	best: 0.5821519 (494)	total: 1m 16s	remaining: 59.9s
561:	learn: 0.5740247	test: 0.5821828	best: 0.5821519 (494)	total: 1m 16s	remaining: 59.8s
562:	learn: 0.5740031	test: 0.5821850	best: 0.5821519 (494)	total: 1m 16s	remaining: 59.7s
563:	learn: 0.573977

644:	learn: 0.5724707	test: 0.5819497	best: 0.5819273 (634)	total: 1m 28s	remaining: 48.5s
645:	learn: 0.5724586	test: 0.5819518	best: 0.5819273 (634)	total: 1m 28s	remaining: 48.3s
646:	learn: 0.5724413	test: 0.5819544	best: 0.5819273 (634)	total: 1m 28s	remaining: 48.2s
647:	learn: 0.5724319	test: 0.5819481	best: 0.5819273 (634)	total: 1m 28s	remaining: 48.1s
648:	learn: 0.5724042	test: 0.5819458	best: 0.5819273 (634)	total: 1m 28s	remaining: 47.9s
649:	learn: 0.5723908	test: 0.5819503	best: 0.5819273 (634)	total: 1m 28s	remaining: 47.8s
650:	learn: 0.5723772	test: 0.5819496	best: 0.5819273 (634)	total: 1m 28s	remaining: 47.7s
651:	learn: 0.5723600	test: 0.5819482	best: 0.5819273 (634)	total: 1m 29s	remaining: 47.5s
652:	learn: 0.5723465	test: 0.5819453	best: 0.5819273 (634)	total: 1m 29s	remaining: 47.4s
653:	learn: 0.5723189	test: 0.5819310	best: 0.5819273 (634)	total: 1m 29s	remaining: 47.3s
654:	learn: 0.5723000	test: 0.5819342	best: 0.5819273 (634)	total: 1m 29s	remaining: 47.2s

735:	learn: 0.5708888	test: 0.5819058	best: 0.5819058 (735)	total: 1m 40s	remaining: 36.1s
736:	learn: 0.5708785	test: 0.5819013	best: 0.5819013 (736)	total: 1m 40s	remaining: 36s
737:	learn: 0.5708532	test: 0.5819018	best: 0.5819013 (736)	total: 1m 40s	remaining: 35.8s
738:	learn: 0.5708377	test: 0.5819029	best: 0.5819013 (736)	total: 1m 41s	remaining: 35.7s
739:	learn: 0.5708241	test: 0.5819050	best: 0.5819013 (736)	total: 1m 41s	remaining: 35.5s
740:	learn: 0.5708008	test: 0.5819108	best: 0.5819013 (736)	total: 1m 41s	remaining: 35.4s
741:	learn: 0.5707861	test: 0.5819125	best: 0.5819013 (736)	total: 1m 41s	remaining: 35.3s
742:	learn: 0.5707807	test: 0.5819114	best: 0.5819013 (736)	total: 1m 41s	remaining: 35.1s
743:	learn: 0.5707717	test: 0.5819084	best: 0.5819013 (736)	total: 1m 41s	remaining: 35s
744:	learn: 0.5707573	test: 0.5819070	best: 0.5819013 (736)	total: 1m 41s	remaining: 34.8s
745:	learn: 0.5707372	test: 0.5819148	best: 0.5819013 (736)	total: 1m 41s	remaining: 34.7s
746

826:	learn: 0.5693641	test: 0.5818539	best: 0.5817997 (812)	total: 1m 53s	remaining: 23.7s
827:	learn: 0.5693513	test: 0.5818635	best: 0.5817997 (812)	total: 1m 53s	remaining: 23.5s
828:	learn: 0.5693359	test: 0.5818686	best: 0.5817997 (812)	total: 1m 53s	remaining: 23.4s
829:	learn: 0.5693133	test: 0.5818713	best: 0.5817997 (812)	total: 1m 53s	remaining: 23.3s
830:	learn: 0.5692946	test: 0.5818624	best: 0.5817997 (812)	total: 1m 53s	remaining: 23.1s
831:	learn: 0.5692842	test: 0.5818668	best: 0.5817997 (812)	total: 1m 53s	remaining: 23s
832:	learn: 0.5692576	test: 0.5818645	best: 0.5817997 (812)	total: 1m 53s	remaining: 22.8s
833:	learn: 0.5692439	test: 0.5818762	best: 0.5817997 (812)	total: 1m 54s	remaining: 22.7s
834:	learn: 0.5692305	test: 0.5818857	best: 0.5817997 (812)	total: 1m 54s	remaining: 22.6s
835:	learn: 0.5692232	test: 0.5818856	best: 0.5817997 (812)	total: 1m 54s	remaining: 22.4s
836:	learn: 0.5692087	test: 0.5818848	best: 0.5817997 (812)	total: 1m 54s	remaining: 22.3s
8

918:	learn: 0.5678746	test: 0.5818894	best: 0.5817997 (812)	total: 2m 6s	remaining: 11.1s
919:	learn: 0.5678610	test: 0.5818911	best: 0.5817997 (812)	total: 2m 6s	remaining: 11s
920:	learn: 0.5678367	test: 0.5818939	best: 0.5817997 (812)	total: 2m 6s	remaining: 10.8s
921:	learn: 0.5678201	test: 0.5818962	best: 0.5817997 (812)	total: 2m 6s	remaining: 10.7s
922:	learn: 0.5678114	test: 0.5818928	best: 0.5817997 (812)	total: 2m 6s	remaining: 10.6s
923:	learn: 0.5677980	test: 0.5818915	best: 0.5817997 (812)	total: 2m 6s	remaining: 10.4s
924:	learn: 0.5677873	test: 0.5818934	best: 0.5817997 (812)	total: 2m 6s	remaining: 10.3s
925:	learn: 0.5677620	test: 0.5818918	best: 0.5817997 (812)	total: 2m 7s	remaining: 10.2s
926:	learn: 0.5677425	test: 0.5818906	best: 0.5817997 (812)	total: 2m 7s	remaining: 10s
927:	learn: 0.5677163	test: 0.5818825	best: 0.5817997 (812)	total: 2m 7s	remaining: 9.88s
928:	learn: 0.5676988	test: 0.5818810	best: 0.5817997 (812)	total: 2m 7s	remaining: 9.74s
929:	learn: 0.

In [27]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Logloss', scale_pos_weight = 4.843983390342822, depth = 8, learning_rate= 0.13711213637660957, iterations = 1000, l2_leaf_reg= 4)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())
print(clf.get_params())

0:	learn: 0.6692085	test: 0.6685806	best: 0.6685806 (0)	total: 239ms	remaining: 3m 58s
1:	learn: 0.6516340	test: 0.6504187	best: 0.6504187 (1)	total: 434ms	remaining: 3m 36s
2:	learn: 0.6387783	test: 0.6371652	best: 0.6371652 (2)	total: 620ms	remaining: 3m 26s
3:	learn: 0.6291229	test: 0.6269612	best: 0.6269612 (3)	total: 824ms	remaining: 3m 25s
4:	learn: 0.6221186	test: 0.6196959	best: 0.6196959 (4)	total: 983ms	remaining: 3m 15s
5:	learn: 0.6164201	test: 0.6142999	best: 0.6142999 (5)	total: 1.17s	remaining: 3m 13s
6:	learn: 0.6121657	test: 0.6098693	best: 0.6098693 (6)	total: 1.35s	remaining: 3m 12s
7:	learn: 0.6089455	test: 0.6068131	best: 0.6068131 (7)	total: 1.54s	remaining: 3m 11s
8:	learn: 0.6063990	test: 0.6042071	best: 0.6042071 (8)	total: 1.74s	remaining: 3m 11s
9:	learn: 0.6044871	test: 0.6021292	best: 0.6021292 (9)	total: 1.92s	remaining: 3m 10s
10:	learn: 0.6029481	test: 0.6005193	best: 0.6005193 (10)	total: 2.1s	remaining: 3m 8s
11:	learn: 0.6015091	test: 0.5992932	best: 

93:	learn: 0.5836114	test: 0.5852137	best: 0.5852137 (93)	total: 16.4s	remaining: 2m 38s
94:	learn: 0.5834977	test: 0.5851858	best: 0.5851858 (94)	total: 16.6s	remaining: 2m 37s
95:	learn: 0.5833789	test: 0.5851254	best: 0.5851254 (95)	total: 16.7s	remaining: 2m 37s
96:	learn: 0.5832834	test: 0.5850804	best: 0.5850804 (96)	total: 16.8s	remaining: 2m 36s
97:	learn: 0.5831913	test: 0.5850501	best: 0.5850501 (97)	total: 17s	remaining: 2m 36s
98:	learn: 0.5831209	test: 0.5850214	best: 0.5850214 (98)	total: 17.2s	remaining: 2m 36s
99:	learn: 0.5830808	test: 0.5850146	best: 0.5850146 (99)	total: 17.4s	remaining: 2m 36s
100:	learn: 0.5829863	test: 0.5849505	best: 0.5849505 (100)	total: 17.6s	remaining: 2m 36s
101:	learn: 0.5828892	test: 0.5849251	best: 0.5849251 (101)	total: 17.7s	remaining: 2m 35s
102:	learn: 0.5828195	test: 0.5848682	best: 0.5848682 (102)	total: 17.9s	remaining: 2m 35s
103:	learn: 0.5827081	test: 0.5847909	best: 0.5847909 (103)	total: 18.1s	remaining: 2m 36s
104:	learn: 0.5

184:	learn: 0.5774983	test: 0.5834742	best: 0.5834742 (184)	total: 32s	remaining: 2m 20s
185:	learn: 0.5774223	test: 0.5834702	best: 0.5834702 (185)	total: 32.2s	remaining: 2m 20s
186:	learn: 0.5773481	test: 0.5834751	best: 0.5834702 (185)	total: 32.4s	remaining: 2m 20s
187:	learn: 0.5772893	test: 0.5834830	best: 0.5834702 (185)	total: 32.5s	remaining: 2m 20s
188:	learn: 0.5772469	test: 0.5834690	best: 0.5834690 (188)	total: 32.7s	remaining: 2m 20s
189:	learn: 0.5771913	test: 0.5834730	best: 0.5834690 (188)	total: 32.9s	remaining: 2m 20s
190:	learn: 0.5771500	test: 0.5834502	best: 0.5834502 (190)	total: 33.1s	remaining: 2m 20s
191:	learn: 0.5771142	test: 0.5834422	best: 0.5834422 (191)	total: 33.3s	remaining: 2m 20s
192:	learn: 0.5770565	test: 0.5834395	best: 0.5834395 (192)	total: 33.5s	remaining: 2m 20s
193:	learn: 0.5769997	test: 0.5834048	best: 0.5834048 (193)	total: 33.7s	remaining: 2m 20s
194:	learn: 0.5769517	test: 0.5834009	best: 0.5834009 (194)	total: 33.9s	remaining: 2m 20s
1

276:	learn: 0.5724119	test: 0.5827432	best: 0.5827326 (275)	total: 48.3s	remaining: 2m 6s
277:	learn: 0.5723770	test: 0.5827320	best: 0.5827320 (277)	total: 48.4s	remaining: 2m 5s
278:	learn: 0.5723312	test: 0.5827313	best: 0.5827313 (278)	total: 48.6s	remaining: 2m 5s
279:	learn: 0.5722728	test: 0.5827303	best: 0.5827303 (279)	total: 48.8s	remaining: 2m 5s
280:	learn: 0.5722382	test: 0.5827198	best: 0.5827198 (280)	total: 48.9s	remaining: 2m 5s
281:	learn: 0.5721842	test: 0.5827209	best: 0.5827198 (280)	total: 49.1s	remaining: 2m 4s
282:	learn: 0.5721174	test: 0.5827160	best: 0.5827160 (282)	total: 49.2s	remaining: 2m 4s
283:	learn: 0.5720372	test: 0.5826957	best: 0.5826957 (283)	total: 49.4s	remaining: 2m 4s
284:	learn: 0.5719681	test: 0.5826812	best: 0.5826812 (284)	total: 49.6s	remaining: 2m 4s
285:	learn: 0.5718942	test: 0.5826822	best: 0.5826812 (284)	total: 49.7s	remaining: 2m 4s
286:	learn: 0.5718334	test: 0.5826850	best: 0.5826812 (284)	total: 49.9s	remaining: 2m 4s
287:	learn

367:	learn: 0.5678929	test: 0.5825860	best: 0.5824848 (337)	total: 1m 4s	remaining: 1m 50s
368:	learn: 0.5678634	test: 0.5825825	best: 0.5824848 (337)	total: 1m 4s	remaining: 1m 49s
369:	learn: 0.5678336	test: 0.5825959	best: 0.5824848 (337)	total: 1m 4s	remaining: 1m 49s
370:	learn: 0.5678105	test: 0.5825964	best: 0.5824848 (337)	total: 1m 4s	remaining: 1m 49s
371:	learn: 0.5677774	test: 0.5826169	best: 0.5824848 (337)	total: 1m 4s	remaining: 1m 49s
372:	learn: 0.5677106	test: 0.5826159	best: 0.5824848 (337)	total: 1m 4s	remaining: 1m 49s
373:	learn: 0.5676620	test: 0.5826202	best: 0.5824848 (337)	total: 1m 5s	remaining: 1m 48s
374:	learn: 0.5675975	test: 0.5826247	best: 0.5824848 (337)	total: 1m 5s	remaining: 1m 48s
375:	learn: 0.5675317	test: 0.5826103	best: 0.5824848 (337)	total: 1m 5s	remaining: 1m 48s
376:	learn: 0.5674646	test: 0.5826175	best: 0.5824848 (337)	total: 1m 5s	remaining: 1m 48s
377:	learn: 0.5674423	test: 0.5826106	best: 0.5824848 (337)	total: 1m 5s	remaining: 1m 48s

...

548:	learn: 0.5599315	test: 0.5825357	best: 0.5824732 (445)	total: 1m 36s	remaining: 1m 18s
549:	learn: 0.5598891	test: 0.5825386	best: 0.5824732 (445)	total: 1m 36s	remaining: 1m 18s
550:	learn: 0.5598465	test: 0.5825370	best: 0.5824732 (445)	total: 1m 36s	remaining: 1m 18s
551:	learn: 0.5597904	test: 0.5825357	best: 0.5824732 (445)	total: 1m 36s	remaining: 1m 18s
552:	learn: 0.5597555	test: 0.5825818	best: 0.5824732 (445)	total: 1m 36s	remaining: 1m 18s
553:	learn: 0.5597051	test: 0.5825914	best: 0.5824732 (445)	total: 1m 37s	remaining: 1m 18s
554:	learn: 0.5596766	test: 0.5825892	best: 0.5824732 (445)	total: 1m 37s	remaining: 1m 17s
555:	learn: 0.5596072	test: 0.5825921	best: 0.5824732 (445)	total: 1m 37s	remaining: 1m 17s
556:	learn: 0.5595722	test: 0.5825873	best: 0.5824732 (445)	total: 1m 37s	remaining: 1m 17s
557:	learn: 0.5595331	test: 0.5825714	best: 0.5824732 (445)	total: 1m 37s	remaining: 1m 17s
558:	learn: 0.5594993	test: 0.5825952	best: 0.5824732 (445)	total: 1m 37s	remain

638:	learn: 0.5562684	test: 0.5827967	best: 0.5824732 (445)	total: 1m 51s	remaining: 1m 3s
639:	learn: 0.5562232	test: 0.5828078	best: 0.5824732 (445)	total: 1m 52s	remaining: 1m 3s
640:	learn: 0.5561749	test: 0.5828042	best: 0.5824732 (445)	total: 1m 52s	remaining: 1m 2s
641:	learn: 0.5561194	test: 0.5828275	best: 0.5824732 (445)	total: 1m 52s	remaining: 1m 2s
642:	learn: 0.5560864	test: 0.5828324	best: 0.5824732 (445)	total: 1m 52s	remaining: 1m 2s
643:	learn: 0.5560492	test: 0.5828266	best: 0.5824732 (445)	total: 1m 52s	remaining: 1m 2s
644:	learn: 0.5560182	test: 0.5828353	best: 0.5824732 (445)	total: 1m 52s	remaining: 1m 2s
645:	learn: 0.5559746	test: 0.5828327	best: 0.5824732 (445)	total: 1m 53s	remaining: 1m 1s
646:	learn: 0.5559308	test: 0.5828286	best: 0.5824732 (445)	total: 1m 53s	remaining: 1m 1s
647:	learn: 0.5558757	test: 0.5828371	best: 0.5824732 (445)	total: 1m 53s	remaining: 1m 1s
648:	learn: 0.5558298	test: 0.5828388	best: 0.5824732 (445)	total: 1m 53s	remaining: 1m 1s

731:	learn: 0.5526450	test: 0.5833867	best: 0.5824732 (445)	total: 2m 7s	remaining: 46.8s
732:	learn: 0.5525794	test: 0.5833885	best: 0.5824732 (445)	total: 2m 7s	remaining: 46.6s
733:	learn: 0.5525571	test: 0.5833861	best: 0.5824732 (445)	total: 2m 8s	remaining: 46.4s
734:	learn: 0.5525258	test: 0.5833873	best: 0.5824732 (445)	total: 2m 8s	remaining: 46.3s
735:	learn: 0.5524889	test: 0.5833887	best: 0.5824732 (445)	total: 2m 8s	remaining: 46.1s
736:	learn: 0.5524349	test: 0.5834010	best: 0.5824732 (445)	total: 2m 8s	remaining: 45.9s
737:	learn: 0.5524097	test: 0.5834096	best: 0.5824732 (445)	total: 2m 8s	remaining: 45.7s
738:	learn: 0.5523940	test: 0.5834150	best: 0.5824732 (445)	total: 2m 8s	remaining: 45.5s
739:	learn: 0.5523098	test: 0.5834117	best: 0.5824732 (445)	total: 2m 9s	remaining: 45.4s
740:	learn: 0.5522604	test: 0.5834145	best: 0.5824732 (445)	total: 2m 9s	remaining: 45.2s
741:	learn: 0.5522312	test: 0.5834209	best: 0.5824732 (445)	total: 2m 9s	remaining: 45s
742:	learn: 

822:	learn: 0.5491050	test: 0.5837057	best: 0.5824732 (445)	total: 2m 23s	remaining: 30.8s
823:	learn: 0.5490681	test: 0.5837042	best: 0.5824732 (445)	total: 2m 23s	remaining: 30.7s
824:	learn: 0.5490428	test: 0.5837196	best: 0.5824732 (445)	total: 2m 23s	remaining: 30.5s
825:	learn: 0.5489929	test: 0.5837226	best: 0.5824732 (445)	total: 2m 23s	remaining: 30.3s
826:	learn: 0.5489478	test: 0.5837200	best: 0.5824732 (445)	total: 2m 24s	remaining: 30.1s
827:	learn: 0.5489216	test: 0.5837239	best: 0.5824732 (445)	total: 2m 24s	remaining: 30s
828:	learn: 0.5488719	test: 0.5837193	best: 0.5824732 (445)	total: 2m 24s	remaining: 29.8s
829:	learn: 0.5488353	test: 0.5837139	best: 0.5824732 (445)	total: 2m 24s	remaining: 29.6s
830:	learn: 0.5487952	test: 0.5837221	best: 0.5824732 (445)	total: 2m 24s	remaining: 29.4s
831:	learn: 0.5487497	test: 0.5837222	best: 0.5824732 (445)	total: 2m 24s	remaining: 29.3s
832:	learn: 0.5487039	test: 0.5837227	best: 0.5824732 (445)	total: 2m 25s	remaining: 29.1s
8

914:	learn: 0.5459017	test: 0.5840338	best: 0.5824732 (445)	total: 2m 39s	remaining: 14.8s
915:	learn: 0.5458657	test: 0.5840402	best: 0.5824732 (445)	total: 2m 39s	remaining: 14.6s
916:	learn: 0.5458176	test: 0.5840553	best: 0.5824732 (445)	total: 2m 39s	remaining: 14.4s
917:	learn: 0.5457945	test: 0.5840585	best: 0.5824732 (445)	total: 2m 39s	remaining: 14.3s
918:	learn: 0.5457709	test: 0.5840579	best: 0.5824732 (445)	total: 2m 39s	remaining: 14.1s
919:	learn: 0.5457317	test: 0.5840657	best: 0.5824732 (445)	total: 2m 40s	remaining: 13.9s
920:	learn: 0.5456948	test: 0.5840749	best: 0.5824732 (445)	total: 2m 40s	remaining: 13.8s
921:	learn: 0.5456755	test: 0.5840748	best: 0.5824732 (445)	total: 2m 40s	remaining: 13.6s
922:	learn: 0.5456615	test: 0.5840762	best: 0.5824732 (445)	total: 2m 40s	remaining: 13.4s
923:	learn: 0.5456406	test: 0.5840781	best: 0.5824732 (445)	total: 2m 40s	remaining: 13.2s
924:	learn: 0.5456094	test: 0.5840912	best: 0.5824732 (445)	total: 2m 40s	remaining: 13.1s

In [28]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Precision', scale_pos_weight = 4.843983390342822, depth = 8, learning_rate= 0.13711213637660957, iterations = 1000, l2_leaf_reg= 4)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())
print(clf.get_params())

0:	learn: 0.6548264	test: 0.6555882	best: 0.6555882 (0)	total: 202ms	remaining: 3m 22s
1:	learn: 0.6520581	test: 0.6547586	best: 0.6555882 (0)	total: 388ms	remaining: 3m 13s
2:	learn: 0.6468356	test: 0.6493588	best: 0.6555882 (0)	total: 572ms	remaining: 3m 10s
3:	learn: 0.6482255	test: 0.6514971	best: 0.6555882 (0)	total: 775ms	remaining: 3m 13s
4:	learn: 0.6491207	test: 0.6519876	best: 0.6555882 (0)	total: 936ms	remaining: 3m 6s
5:	learn: 0.6488685	test: 0.6507057	best: 0.6555882 (0)	total: 1.12s	remaining: 3m 5s
6:	learn: 0.6490935	test: 0.6517101	best: 0.6555882 (0)	total: 1.3s	remaining: 3m 4s
7:	learn: 0.6495989	test: 0.6513940	best: 0.6555882 (0)	total: 1.48s	remaining: 3m 3s
8:	learn: 0.6494035	test: 0.6512135	best: 0.6555882 (0)	total: 1.68s	remaining: 3m 4s
9:	learn: 0.6489489	test: 0.6510925	best: 0.6555882 (0)	total: 1.86s	remaining: 3m 4s
10:	learn: 0.6496757	test: 0.6516502	best: 0.6555882 (0)	total: 2.04s	remaining: 3m 3s
11:	learn: 0.6503095	test: 0.6521543	best: 0.65558

93:	learn: 0.6621536	test: 0.6601622	best: 0.6601845 (92)	total: 16.6s	remaining: 2m 40s
94:	learn: 0.6622814	test: 0.6602643	best: 0.6602643 (94)	total: 16.7s	remaining: 2m 39s
95:	learn: 0.6623396	test: 0.6602196	best: 0.6602643 (94)	total: 16.9s	remaining: 2m 38s
96:	learn: 0.6623633	test: 0.6602161	best: 0.6602643 (94)	total: 17s	remaining: 2m 38s
97:	learn: 0.6624118	test: 0.6602107	best: 0.6602643 (94)	total: 17.2s	remaining: 2m 38s
98:	learn: 0.6625004	test: 0.6602280	best: 0.6602643 (94)	total: 17.4s	remaining: 2m 38s
99:	learn: 0.6625427	test: 0.6602238	best: 0.6602643 (94)	total: 17.6s	remaining: 2m 38s
100:	learn: 0.6626301	test: 0.6604072	best: 0.6604072 (100)	total: 17.8s	remaining: 2m 38s
101:	learn: 0.6626326	test: 0.6603786	best: 0.6604072 (100)	total: 17.9s	remaining: 2m 37s
102:	learn: 0.6626919	test: 0.6604232	best: 0.6604232 (102)	total: 18.1s	remaining: 2m 37s
103:	learn: 0.6628719	test: 0.6605338	best: 0.6605338 (103)	total: 18.3s	remaining: 2m 37s
104:	learn: 0.6

185:	learn: 0.6670925	test: 0.6626711	best: 0.6627327 (182)	total: 32.6s	remaining: 2m 22s
186:	learn: 0.6671692	test: 0.6626994	best: 0.6627327 (182)	total: 32.8s	remaining: 2m 22s
187:	learn: 0.6671775	test: 0.6627140	best: 0.6627327 (182)	total: 32.9s	remaining: 2m 22s
188:	learn: 0.6671854	test: 0.6627207	best: 0.6627327 (182)	total: 33.1s	remaining: 2m 22s
189:	learn: 0.6672050	test: 0.6627119	best: 0.6627327 (182)	total: 33.3s	remaining: 2m 21s
190:	learn: 0.6672146	test: 0.6627080	best: 0.6627327 (182)	total: 33.4s	remaining: 2m 21s
191:	learn: 0.6672575	test: 0.6627352	best: 0.6627352 (191)	total: 33.6s	remaining: 2m 21s
192:	learn: 0.6672970	test: 0.6627276	best: 0.6627352 (191)	total: 33.8s	remaining: 2m 21s
193:	learn: 0.6673301	test: 0.6627309	best: 0.6627352 (191)	total: 34s	remaining: 2m 21s
194:	learn: 0.6673170	test: 0.6627044	best: 0.6627352 (191)	total: 34.2s	remaining: 2m 21s
195:	learn: 0.6673464	test: 0.6627183	best: 0.6627352 (191)	total: 34.3s	remaining: 2m 20s
1

276:	learn: 0.6705344	test: 0.6636957	best: 0.6636957 (276)	total: 48.3s	remaining: 2m 6s
277:	learn: 0.6705196	test: 0.6637406	best: 0.6637406 (277)	total: 48.5s	remaining: 2m 5s
278:	learn: 0.6705841	test: 0.6636898	best: 0.6637406 (277)	total: 48.6s	remaining: 2m 5s
279:	learn: 0.6705956	test: 0.6636918	best: 0.6637406 (277)	total: 48.8s	remaining: 2m 5s
280:	learn: 0.6705869	test: 0.6637799	best: 0.6637799 (280)	total: 48.9s	remaining: 2m 5s
281:	learn: 0.6705978	test: 0.6637327	best: 0.6637799 (280)	total: 49.1s	remaining: 2m 4s
282:	learn: 0.6706275	test: 0.6636647	best: 0.6637799 (280)	total: 49.3s	remaining: 2m 4s
283:	learn: 0.6706768	test: 0.6636498	best: 0.6637799 (280)	total: 49.4s	remaining: 2m 4s
284:	learn: 0.6707559	test: 0.6637017	best: 0.6637799 (280)	total: 49.6s	remaining: 2m 4s
285:	learn: 0.6708046	test: 0.6637551	best: 0.6637799 (280)	total: 49.8s	remaining: 2m 4s
286:	learn: 0.6708604	test: 0.6636712	best: 0.6637799 (280)	total: 49.9s	remaining: 2m 4s
287:	learn

367:	learn: 0.6735744	test: 0.6641312	best: 0.6642990 (362)	total: 1m 3s	remaining: 1m 49s
368:	learn: 0.6735714	test: 0.6641833	best: 0.6642990 (362)	total: 1m 4s	remaining: 1m 49s
369:	learn: 0.6736514	test: 0.6641714	best: 0.6642990 (362)	total: 1m 4s	remaining: 1m 49s
370:	learn: 0.6736048	test: 0.6642016	best: 0.6642990 (362)	total: 1m 4s	remaining: 1m 49s
371:	learn: 0.6736443	test: 0.6641971	best: 0.6642990 (362)	total: 1m 4s	remaining: 1m 49s
372:	learn: 0.6736782	test: 0.6642038	best: 0.6642990 (362)	total: 1m 4s	remaining: 1m 48s
373:	learn: 0.6737000	test: 0.6642253	best: 0.6642990 (362)	total: 1m 4s	remaining: 1m 48s
374:	learn: 0.6737130	test: 0.6641968	best: 0.6642990 (362)	total: 1m 5s	remaining: 1m 48s
375:	learn: 0.6738094	test: 0.6642317	best: 0.6642990 (362)	total: 1m 5s	remaining: 1m 48s
376:	learn: 0.6738930	test: 0.6641659	best: 0.6642990 (362)	total: 1m 5s	remaining: 1m 48s
377:	learn: 0.6738304	test: 0.6641608	best: 0.6642990 (362)	total: 1m 5s	remaining: 1m 48s

...

549:	learn: 0.6793658	test: 0.6652941	best: 0.6654049 (541)	total: 1m 36s	remaining: 1m 18s
550:	learn: 0.6793848	test: 0.6653117	best: 0.6654049 (541)	total: 1m 36s	remaining: 1m 18s
551:	learn: 0.6794454	test: 0.6653428	best: 0.6654049 (541)	total: 1m 36s	remaining: 1m 18s
552:	learn: 0.6794910	test: 0.6653905	best: 0.6654049 (541)	total: 1m 36s	remaining: 1m 18s
553:	learn: 0.6795536	test: 0.6653786	best: 0.6654049 (541)	total: 1m 36s	remaining: 1m 18s
554:	learn: 0.6795481	test: 0.6653988	best: 0.6654049 (541)	total: 1m 37s	remaining: 1m 17s
555:	learn: 0.6795767	test: 0.6654310	best: 0.6654310 (555)	total: 1m 37s	remaining: 1m 17s
556:	learn: 0.6795945	test: 0.6655008	best: 0.6655008 (556)	total: 1m 37s	remaining: 1m 17s
557:	learn: 0.6796254	test: 0.6654994	best: 0.6655008 (556)	total: 1m 37s	remaining: 1m 17s
558:	learn: 0.6797042	test: 0.6654976	best: 0.6655008 (556)	total: 1m 37s	remaining: 1m 17s
559:	learn: 0.6798125	test: 0.6654702	best: 0.6655008 (556)	total: 1m 38s	remain

639:	learn: 0.6819485	test: 0.6661060	best: 0.6661060 (639)	total: 1m 52s	remaining: 1m 3s
640:	learn: 0.6819835	test: 0.6661043	best: 0.6661060 (639)	total: 1m 52s	remaining: 1m 2s
641:	learn: 0.6819914	test: 0.6660871	best: 0.6661060 (639)	total: 1m 52s	remaining: 1m 2s
642:	learn: 0.6820526	test: 0.6661327	best: 0.6661327 (642)	total: 1m 52s	remaining: 1m 2s
643:	learn: 0.6820655	test: 0.6661099	best: 0.6661327 (642)	total: 1m 53s	remaining: 1m 2s
644:	learn: 0.6821121	test: 0.6661236	best: 0.6661327 (642)	total: 1m 53s	remaining: 1m 2s
645:	learn: 0.6820967	test: 0.6661314	best: 0.6661327 (642)	total: 1m 53s	remaining: 1m 2s
646:	learn: 0.6821374	test: 0.6661382	best: 0.6661382 (646)	total: 1m 53s	remaining: 1m 1s
647:	learn: 0.6821029	test: 0.6661187	best: 0.6661382 (646)	total: 1m 53s	remaining: 1m 1s
648:	learn: 0.6821083	test: 0.6661047	best: 0.6661382 (646)	total: 1m 53s	remaining: 1m 1s
649:	learn: 0.6821230	test: 0.6661257	best: 0.6661382 (646)	total: 1m 54s	remaining: 1m 1s

731:	learn: 0.6844664	test: 0.6665663	best: 0.6666028 (728)	total: 2m 8s	remaining: 47s
732:	learn: 0.6845348	test: 0.6665375	best: 0.6666028 (728)	total: 2m 8s	remaining: 46.8s
733:	learn: 0.6845207	test: 0.6666342	best: 0.6666342 (733)	total: 2m 8s	remaining: 46.6s
734:	learn: 0.6845354	test: 0.6666480	best: 0.6666480 (734)	total: 2m 8s	remaining: 46.5s
735:	learn: 0.6845798	test: 0.6666727	best: 0.6666727 (735)	total: 2m 9s	remaining: 46.3s
736:	learn: 0.6846407	test: 0.6666859	best: 0.6666859 (736)	total: 2m 9s	remaining: 46.1s
737:	learn: 0.6846287	test: 0.6666475	best: 0.6666859 (736)	total: 2m 9s	remaining: 45.9s
738:	learn: 0.6846284	test: 0.6666380	best: 0.6666859 (736)	total: 2m 9s	remaining: 45.8s
739:	learn: 0.6847254	test: 0.6666359	best: 0.6666859 (736)	total: 2m 9s	remaining: 45.6s
740:	learn: 0.6847500	test: 0.6666538	best: 0.6666859 (736)	total: 2m 9s	remaining: 45.4s
741:	learn: 0.6847783	test: 0.6666316	best: 0.6666859 (736)	total: 2m 10s	remaining: 45.2s
742:	learn:

822:	learn: 0.6870082	test: 0.6665796	best: 0.6666859 (736)	total: 2m 24s	remaining: 31.1s
823:	learn: 0.6870453	test: 0.6665713	best: 0.6666859 (736)	total: 2m 24s	remaining: 30.9s
824:	learn: 0.6870765	test: 0.6665777	best: 0.6666859 (736)	total: 2m 24s	remaining: 30.7s
825:	learn: 0.6871067	test: 0.6665740	best: 0.6666859 (736)	total: 2m 25s	remaining: 30.6s
826:	learn: 0.6872150	test: 0.6665968	best: 0.6666859 (736)	total: 2m 25s	remaining: 30.4s
827:	learn: 0.6872492	test: 0.6665562	best: 0.6666859 (736)	total: 2m 25s	remaining: 30.2s
828:	learn: 0.6872207	test: 0.6666322	best: 0.6666859 (736)	total: 2m 25s	remaining: 30s
829:	learn: 0.6872701	test: 0.6666693	best: 0.6666859 (736)	total: 2m 25s	remaining: 29.9s
830:	learn: 0.6872826	test: 0.6667028	best: 0.6667028 (830)	total: 2m 26s	remaining: 29.7s
831:	learn: 0.6873242	test: 0.6667167	best: 0.6667167 (831)	total: 2m 26s	remaining: 29.5s
832:	learn: 0.6873736	test: 0.6667587	best: 0.6667587 (832)	total: 2m 26s	remaining: 29.3s
8

914:	learn: 0.6895506	test: 0.6672154	best: 0.6672154 (914)	total: 2m 40s	remaining: 15s
915:	learn: 0.6895873	test: 0.6672592	best: 0.6672592 (915)	total: 2m 41s	remaining: 14.8s
916:	learn: 0.6896469	test: 0.6671835	best: 0.6672592 (915)	total: 2m 41s	remaining: 14.6s
917:	learn: 0.6896962	test: 0.6671744	best: 0.6672592 (915)	total: 2m 41s	remaining: 14.4s
918:	learn: 0.6897070	test: 0.6671608	best: 0.6672592 (915)	total: 2m 41s	remaining: 14.2s
919:	learn: 0.6896852	test: 0.6671789	best: 0.6672592 (915)	total: 2m 41s	remaining: 14.1s
920:	learn: 0.6897214	test: 0.6671435	best: 0.6672592 (915)	total: 2m 41s	remaining: 13.9s
921:	learn: 0.6897645	test: 0.6671634	best: 0.6672592 (915)	total: 2m 42s	remaining: 13.7s
922:	learn: 0.6898114	test: 0.6671573	best: 0.6672592 (915)	total: 2m 42s	remaining: 13.5s
923:	learn: 0.6898954	test: 0.6672065	best: 0.6672592 (915)	total: 2m 42s	remaining: 13.4s
924:	learn: 0.6899563	test: 0.6672072	best: 0.6672592 (915)	total: 2m 42s	remaining: 13.2s
9

In [29]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Recall', scale_pos_weight = 4.843983390342822, depth = 8, learning_rate= 0.13711213637660957, iterations = 1000, l2_leaf_reg= 4)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X2, y2))
print(clf.get_best_score())
print(clf.get_params())

0:	learn: 0.7060398	test: 0.7181417	best: 0.7181417 (0)	total: 225ms	remaining: 3m 44s
1:	learn: 0.7261747	test: 0.7298442	best: 0.7298442 (1)	total: 419ms	remaining: 3m 29s
2:	learn: 0.7541093	test: 0.7555072	best: 0.7555072 (2)	total: 608ms	remaining: 3m 22s
3:	learn: 0.7489194	test: 0.7505087	best: 0.7555072 (2)	total: 816ms	remaining: 3m 23s
4:	learn: 0.7508086	test: 0.7533196	best: 0.7555072 (2)	total: 980ms	remaining: 3m 14s
5:	learn: 0.7545210	test: 0.7587768	best: 0.7587768 (5)	total: 1.16s	remaining: 3m 12s
6:	learn: 0.7569395	test: 0.7598001	best: 0.7598001 (6)	total: 1.35s	remaining: 3m 12s
7:	learn: 0.7561824	test: 0.7601176	best: 0.7601176 (7)	total: 1.54s	remaining: 3m 11s
8:	learn: 0.7598065	test: 0.7627051	best: 0.7627051 (8)	total: 1.75s	remaining: 3m 12s
9:	learn: 0.7610489	test: 0.7633049	best: 0.7633049 (9)	total: 1.93s	remaining: 3m 10s
10:	learn: 0.7595786	test: 0.7617524	best: 0.7633049 (9)	total: 2.1s	remaining: 3m 9s
11:	learn: 0.7592478	test: 0.7620582	best: 0

93:	learn: 0.7620707	test: 0.7631991	best: 0.7642458 (32)	total: 16.6s	remaining: 2m 40s
94:	learn: 0.7621221	test: 0.7630462	best: 0.7642458 (32)	total: 16.8s	remaining: 2m 39s
95:	learn: 0.7622104	test: 0.7625639	best: 0.7642458 (32)	total: 16.9s	remaining: 2m 39s
96:	learn: 0.7622912	test: 0.7626463	best: 0.7642458 (32)	total: 17s	remaining: 2m 38s
97:	learn: 0.7622839	test: 0.7624111	best: 0.7642458 (32)	total: 17.2s	remaining: 2m 38s
98:	learn: 0.7623059	test: 0.7625404	best: 0.7642458 (32)	total: 17.4s	remaining: 2m 38s
99:	learn: 0.7623280	test: 0.7624934	best: 0.7642458 (32)	total: 17.6s	remaining: 2m 38s
100:	learn: 0.7623280	test: 0.7623993	best: 0.7642458 (32)	total: 17.8s	remaining: 2m 38s
101:	learn: 0.7623574	test: 0.7625051	best: 0.7642458 (32)	total: 17.9s	remaining: 2m 37s
102:	learn: 0.7623868	test: 0.7624346	best: 0.7642458 (32)	total: 18.1s	remaining: 2m 37s
103:	learn: 0.7626220	test: 0.7627874	best: 0.7642458 (32)	total: 18.3s	remaining: 2m 37s
104:	learn: 0.76254

185:	learn: 0.7669960	test: 0.7603881	best: 0.7642458 (32)	total: 32.5s	remaining: 2m 22s
186:	learn: 0.7669813	test: 0.7603411	best: 0.7642458 (32)	total: 32.7s	remaining: 2m 22s
187:	learn: 0.7671283	test: 0.7603528	best: 0.7642458 (32)	total: 32.9s	remaining: 2m 22s
188:	learn: 0.7670401	test: 0.7605998	best: 0.7642458 (32)	total: 33.1s	remaining: 2m 21s
189:	learn: 0.7669739	test: 0.7605410	best: 0.7642458 (32)	total: 33.2s	remaining: 2m 21s
190:	learn: 0.7671136	test: 0.7607998	best: 0.7642458 (32)	total: 33.4s	remaining: 2m 21s
191:	learn: 0.7671430	test: 0.7608350	best: 0.7642458 (32)	total: 33.6s	remaining: 2m 21s
192:	learn: 0.7671063	test: 0.7607998	best: 0.7642458 (32)	total: 33.7s	remaining: 2m 21s
193:	learn: 0.7672753	test: 0.7606821	best: 0.7642458 (32)	total: 33.9s	remaining: 2m 21s
194:	learn: 0.7672606	test: 0.7608115	best: 0.7642458 (32)	total: 34.1s	remaining: 2m 20s
195:	learn: 0.7673195	test: 0.7606821	best: 0.7642458 (32)	total: 34.3s	remaining: 2m 20s
196:	learn

277:	learn: 0.7704805	test: 0.7591061	best: 0.7642458 (32)	total: 48.5s	remaining: 2m 6s
278:	learn: 0.7706128	test: 0.7591297	best: 0.7642458 (32)	total: 48.7s	remaining: 2m 5s
279:	learn: 0.7704584	test: 0.7591414	best: 0.7642458 (32)	total: 48.8s	remaining: 2m 5s
280:	learn: 0.7705393	test: 0.7593884	best: 0.7642458 (32)	total: 49s	remaining: 2m 5s
281:	learn: 0.7704878	test: 0.7592708	best: 0.7642458 (32)	total: 49.2s	remaining: 2m 5s
282:	learn: 0.7705760	test: 0.7590826	best: 0.7642458 (32)	total: 49.3s	remaining: 2m 4s
283:	learn: 0.7707231	test: 0.7590944	best: 0.7642458 (32)	total: 49.5s	remaining: 2m 4s
284:	learn: 0.7708260	test: 0.7591414	best: 0.7642458 (32)	total: 49.7s	remaining: 2m 4s
285:	learn: 0.7708848	test: 0.7591649	best: 0.7642458 (32)	total: 49.8s	remaining: 2m 4s
286:	learn: 0.7710612	test: 0.7589180	best: 0.7642458 (32)	total: 50s	remaining: 2m 4s
287:	learn: 0.7711421	test: 0.7588356	best: 0.7642458 (32)	total: 50.2s	remaining: 2m 4s
288:	learn: 0.7711715	tes

369:	learn: 0.7737077	test: 0.7579418	best: 0.7642458 (32)	total: 1m 4s	remaining: 1m 49s
370:	learn: 0.7736562	test: 0.7580829	best: 0.7642458 (32)	total: 1m 4s	remaining: 1m 49s
371:	learn: 0.7737077	test: 0.7581300	best: 0.7642458 (32)	total: 1m 4s	remaining: 1m 49s
372:	learn: 0.7737518	test: 0.7580712	best: 0.7642458 (32)	total: 1m 4s	remaining: 1m 49s
373:	learn: 0.7738253	test: 0.7583652	best: 0.7642458 (32)	total: 1m 5s	remaining: 1m 49s
374:	learn: 0.7738179	test: 0.7584122	best: 0.7642458 (32)	total: 1m 5s	remaining: 1m 48s
375:	learn: 0.7737371	test: 0.7576713	best: 0.7642458 (32)	total: 1m 5s	remaining: 1m 48s
376:	learn: 0.7738620	test: 0.7576830	best: 0.7642458 (32)	total: 1m 5s	remaining: 1m 48s
377:	learn: 0.7738988	test: 0.7578242	best: 0.7642458 (32)	total: 1m 5s	remaining: 1m 48s
378:	learn: 0.7739797	test: 0.7576830	best: 0.7642458 (32)	total: 1m 6s	remaining: 1m 48s
379:	learn: 0.7739870	test: 0.7581417	best: 0.7642458 (32)	total: 1m 6s	remaining: 1m 48s
380:	learn

...

551:	learn: 0.7796768	test: 0.7526727	best: 0.7642458 (32)	total: 1m 37s	remaining: 1m 18s
552:	learn: 0.7797209	test: 0.7526845	best: 0.7642458 (32)	total: 1m 37s	remaining: 1m 18s
553:	learn: 0.7798753	test: 0.7528374	best: 0.7642458 (32)	total: 1m 37s	remaining: 1m 18s
554:	learn: 0.7799488	test: 0.7528609	best: 0.7642458 (32)	total: 1m 37s	remaining: 1m 18s
555:	learn: 0.7800076	test: 0.7529080	best: 0.7642458 (32)	total: 1m 37s	remaining: 1m 18s
556:	learn: 0.7799415	test: 0.7527668	best: 0.7642458 (32)	total: 1m 38s	remaining: 1m 17s
557:	learn: 0.7800076	test: 0.7526375	best: 0.7642458 (32)	total: 1m 38s	remaining: 1m 17s
558:	learn: 0.7799635	test: 0.7526610	best: 0.7642458 (32)	total: 1m 38s	remaining: 1m 17s
559:	learn: 0.7800150	test: 0.7525081	best: 0.7642458 (32)	total: 1m 38s	remaining: 1m 17s
560:	learn: 0.7800885	test: 0.7525551	best: 0.7642458 (32)	total: 1m 38s	remaining: 1m 17s
561:	learn: 0.7801179	test: 0.7525198	best: 0.7642458 (32)	total: 1m 38s	remaining: 1m 17s

642:	learn: 0.7829187	test: 0.7497442	best: 0.7642458 (32)	total: 1m 53s	remaining: 1m 2s
643:	learn: 0.7828599	test: 0.7497560	best: 0.7642458 (32)	total: 1m 53s	remaining: 1m 2s
644:	learn: 0.7829334	test: 0.7496031	best: 0.7642458 (32)	total: 1m 53s	remaining: 1m 2s
645:	learn: 0.7830584	test: 0.7495560	best: 0.7642458 (32)	total: 1m 53s	remaining: 1m 2s
646:	learn: 0.7832275	test: 0.7495560	best: 0.7642458 (32)	total: 1m 53s	remaining: 1m 2s
647:	learn: 0.7832275	test: 0.7494619	best: 0.7642458 (32)	total: 1m 54s	remaining: 1m 1s
648:	learn: 0.7831613	test: 0.7494384	best: 0.7642458 (32)	total: 1m 54s	remaining: 1m 1s
649:	learn: 0.7830363	test: 0.7494384	best: 0.7642458 (32)	total: 1m 54s	remaining: 1m 1s
650:	learn: 0.7829996	test: 0.7493796	best: 0.7642458 (32)	total: 1m 54s	remaining: 1m 1s
651:	learn: 0.7829775	test: 0.7491679	best: 0.7642458 (32)	total: 1m 54s	remaining: 1m 1s
652:	learn: 0.7829922	test: 0.7492502	best: 0.7642458 (32)	total: 1m 54s	remaining: 1m 1s
653:	learn

735:	learn: 0.7857783	test: 0.7469685	best: 0.7642458 (32)	total: 2m 9s	remaining: 46.5s
736:	learn: 0.7858886	test: 0.7470038	best: 0.7642458 (32)	total: 2m 9s	remaining: 46.3s
737:	learn: 0.7859548	test: 0.7469450	best: 0.7642458 (32)	total: 2m 9s	remaining: 46.1s
738:	learn: 0.7860577	test: 0.7469215	best: 0.7642458 (32)	total: 2m 10s	remaining: 45.9s
739:	learn: 0.7860283	test: 0.7468156	best: 0.7642458 (32)	total: 2m 10s	remaining: 45.8s
740:	learn: 0.7860797	test: 0.7468156	best: 0.7642458 (32)	total: 2m 10s	remaining: 45.6s
741:	learn: 0.7860724	test: 0.7468744	best: 0.7642458 (32)	total: 2m 10s	remaining: 45.4s
742:	learn: 0.7862341	test: 0.7468274	best: 0.7642458 (32)	total: 2m 10s	remaining: 45.2s
743:	learn: 0.7862562	test: 0.7469450	best: 0.7642458 (32)	total: 2m 10s	remaining: 45s
744:	learn: 0.7863150	test: 0.7469215	best: 0.7642458 (32)	total: 2m 11s	remaining: 44.9s
745:	learn: 0.7864032	test: 0.7469450	best: 0.7642458 (32)	total: 2m 11s	remaining: 44.7s
746:	learn: 0.7

828:	learn: 0.7889541	test: 0.7454631	best: 0.7642458 (32)	total: 2m 25s	remaining: 30.1s
829:	learn: 0.7889541	test: 0.7456630	best: 0.7642458 (32)	total: 2m 26s	remaining: 29.9s
830:	learn: 0.7889467	test: 0.7456278	best: 0.7642458 (32)	total: 2m 26s	remaining: 29.8s
831:	learn: 0.7889982	test: 0.7456983	best: 0.7642458 (32)	total: 2m 26s	remaining: 29.6s
832:	learn: 0.7892261	test: 0.7457101	best: 0.7642458 (32)	total: 2m 26s	remaining: 29.4s
833:	learn: 0.7892334	test: 0.7456748	best: 0.7642458 (32)	total: 2m 26s	remaining: 29.2s
834:	learn: 0.7892040	test: 0.7457101	best: 0.7642458 (32)	total: 2m 27s	remaining: 29.1s
835:	learn: 0.7892261	test: 0.7456983	best: 0.7642458 (32)	total: 2m 27s	remaining: 28.9s
836:	learn: 0.7893290	test: 0.7457689	best: 0.7642458 (32)	total: 2m 27s	remaining: 28.7s
837:	learn: 0.7893951	test: 0.7456866	best: 0.7642458 (32)	total: 2m 27s	remaining: 28.5s
838:	learn: 0.7895054	test: 0.7456278	best: 0.7642458 (32)	total: 2m 27s	remaining: 28.4s
839:	learn

920:	learn: 0.7917255	test: 0.7439224	best: 0.7642458 (32)	total: 2m 41s	remaining: 13.9s
921:	learn: 0.7918504	test: 0.7440047	best: 0.7642458 (32)	total: 2m 41s	remaining: 13.7s
922:	learn: 0.7919607	test: 0.7440400	best: 0.7642458 (32)	total: 2m 42s	remaining: 13.5s
923:	learn: 0.7920269	test: 0.7439341	best: 0.7642458 (32)	total: 2m 42s	remaining: 13.3s
924:	learn: 0.7920636	test: 0.7439106	best: 0.7642458 (32)	total: 2m 42s	remaining: 13.2s
925:	learn: 0.7921151	test: 0.7439224	best: 0.7642458 (32)	total: 2m 42s	remaining: 13s
926:	learn: 0.7920930	test: 0.7438518	best: 0.7642458 (32)	total: 2m 42s	remaining: 12.8s
927:	learn: 0.7920342	test: 0.7435931	best: 0.7642458 (32)	total: 2m 42s	remaining: 12.6s
928:	learn: 0.7921739	test: 0.7435931	best: 0.7642458 (32)	total: 2m 43s	remaining: 12.5s
929:	learn: 0.7921739	test: 0.7435813	best: 0.7642458 (32)	total: 2m 43s	remaining: 12.3s
930:	learn: 0.7923062	test: 0.7435225	best: 0.7642458 (32)	total: 2m 43s	remaining: 12.1s
931:	learn: 

In [13]:
train_1mm = pd.read_csv('cleanedTraining.csv', header=0)
train_1mm.drop(train_1mm.columns[[0]], axis = 1, inplace = True) 

c:\users\shenhao\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (28,30,31,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
train_1mm.dtypes

click                  int64
hour                   int64
C1                     int64
banner_pos             int64
site_id               object
site_domain           object
site_category         object
app_id                object
app_domain            object
app_category          object
device_id             object
device_ip             object
device_model          object
device_type            int64
device_conn_type       int64
C14                    int64
C15                    int64
C16                    int64
C17                    int64
C18                    int64
C19                    int64
C20                    int64
C21                    int64
avg_clicks           float64
new_site_id           object
count                  int64
new_site_domain       object
new_site_category     object
new_app_id            object
new_app_domain        object
new_app_category      object
new_device_id         object
new_device_ip         object
new_device_model      object
new_C14       

In [15]:
train_1mm.drop(train_1mm.columns[[4,5,6,7,8,9,10,11,12,15,18,20,21,22,23,25]], axis = 1, inplace = True) 

In [16]:
train_1mm.dtypes

click                 int64
hour                  int64
C1                    int64
banner_pos            int64
device_type           int64
device_conn_type      int64
C15                   int64
C16                   int64
C18                   int64
new_site_id          object
new_site_domain      object
new_site_category    object
new_app_id           object
new_app_domain       object
new_app_category     object
new_device_id        object
new_device_ip        object
new_device_model     object
new_C14              object
new_C17              object
new_C19              object
new_C20              object
new_C21              object
dtype: object

In [17]:
train_1mm

,click,hour,C1,banner_pos,device_type,device_conn_type,C15,C16,C18,new_site_id,...,new_app_domain,new_app_category,new_device_id,new_device_ip,new_device_model,new_C14,new_C17,new_C19,new_C20,new_C21
0,0,9,1005,0,1,2,320,50,0,2,...,2,1,0,low_freq,2,2,1,2,3,2
1,0,5,1005,1,1,0,320,50,2,3,...,2,2,0,low_freq,2,5,3,3,2,3
2,1,17,1005,1,1,0,320,50,2,4,...,2,2,0,low_freq,2,6,3,3,3,3
3,0,16,1005,0,1,3,320,50,1,2,...,2,1,low_freq,low_freq,2,1,0,0,0,0
4,1,7,1005,1,1,0,320,50,3,2,...,2,2,0,low_freq,2,3,1,3,2,3
5,0,17,1005,1,1,0,320,50,3,0,...,2,2,0,low_freq,2,3,1,3,3,3
6,0,9,1005,0,1,0,320,50,0,3,...,2,2,0,low_freq,1,3,1,3,1,2
7,0,4,1005,0,1,0,320,50,0,1,...,2,2,0,low_freq,2,2,1,2,2,2
8,1,17,1005,1,1,0,320,50,3,4,...,2,2,0,low_freq,3,6,3,3,3,3
9,0,4,1005,0,1,0,320,50,3,2,...,2,1,low_freq,low_freq,3,3,2,3,3,3


In [21]:
X = train_1mm.drop(['click'], axis=1)
y = train_1mm.click
categorical_feature_indices = np.where(X.dtypes == np.object)[0]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [22]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Logloss')
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X_val, y_val))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.5940022	test: 0.5938071	best: 0.5938071 (0)	total: 132ms	remaining: 2m 11s
1:	learn: 0.5290336	test: 0.5286846	best: 0.5286846 (1)	total: 268ms	remaining: 2m 13s
2:	learn: 0.4893177	test: 0.4887346	best: 0.4887346 (2)	total: 413ms	remaining: 2m 17s
3:	learn: 0.4611568	test: 0.4605294	best: 0.4605294 (3)	total: 562ms	remaining: 2m 19s
4:	learn: 0.4443682	test: 0.4436890	best: 0.4436890 (4)	total: 712ms	remaining: 2m 21s
5:	learn: 0.4324184	test: 0.4316816	best: 0.4316816 (5)	total: 860ms	remaining: 2m 22s
6:	learn: 0.4240268	test: 0.4232334	best: 0.4232334 (6)	total: 1s	remaining: 2m 21s
7:	learn: 0.4182387	test: 0.4174384	best: 0.4174384 (7)	total: 1.13s	remaining: 2m 19s
8:	learn: 0.4134593	test: 0.4126172	best: 0.4126172 (8)	total: 1.25s	remaining: 2m 18s
9:	learn: 0.4101874	test: 0.4093450	best: 0.4093450 (9)	total: 1.38s	remaining: 2m 16s
10:	learn: 0.4077014	test: 0.4068375	best: 0.4068375 (10)	total: 1.52s	remaining: 2m 16s
11:	learn: 0.4

94:	learn: 0.3977432	test: 0.3975011	best: 0.3975011 (94)	total: 13.6s	remaining: 2m 9s
95:	learn: 0.3977392	test: 0.3975011	best: 0.3975011 (94)	total: 13.7s	remaining: 2m 8s
96:	learn: 0.3977057	test: 0.3974796	best: 0.3974796 (96)	total: 13.8s	remaining: 2m 8s
97:	learn: 0.3976837	test: 0.3974687	best: 0.3974687 (97)	total: 14s	remaining: 2m 8s
98:	learn: 0.3976714	test: 0.3974623	best: 0.3974623 (98)	total: 14.1s	remaining: 2m 8s
99:	learn: 0.3976517	test: 0.3974375	best: 0.3974375 (99)	total: 14.3s	remaining: 2m 8s
100:	learn: 0.3976377	test: 0.3974289	best: 0.3974289 (100)	total: 14.4s	remaining: 2m 8s
101:	learn: 0.3976282	test: 0.3974275	best: 0.3974275 (101)	total: 14.5s	remaining: 2m 7s
102:	learn: 0.3975953	test: 0.3974050	best: 0.3974050 (102)	total: 14.7s	remaining: 2m 7s
103:	learn: 0.3975900	test: 0.3974058	best: 0.3974050 (102)	total: 14.8s	remaining: 2m 7s
104:	learn: 0.3975696	test: 0.3973888	best: 0.3973888 (104)	total: 14.9s	remaining: 2m 7s
105:	learn: 0.3975608	te

186:	learn: 0.3965524	test: 0.3968657	best: 0.3968643 (185)	total: 25.9s	remaining: 1m 52s
187:	learn: 0.3965377	test: 0.3968643	best: 0.3968643 (187)	total: 26.1s	remaining: 1m 52s
188:	learn: 0.3965221	test: 0.3968537	best: 0.3968537 (188)	total: 26.2s	remaining: 1m 52s
189:	learn: 0.3965186	test: 0.3968578	best: 0.3968537 (188)	total: 26.4s	remaining: 1m 52s
190:	learn: 0.3965102	test: 0.3968504	best: 0.3968504 (190)	total: 26.5s	remaining: 1m 52s
191:	learn: 0.3964966	test: 0.3968517	best: 0.3968504 (190)	total: 26.6s	remaining: 1m 52s
192:	learn: 0.3964954	test: 0.3968514	best: 0.3968504 (190)	total: 26.7s	remaining: 1m 51s
193:	learn: 0.3964890	test: 0.3968436	best: 0.3968436 (193)	total: 26.9s	remaining: 1m 51s
194:	learn: 0.3964864	test: 0.3968389	best: 0.3968389 (194)	total: 27s	remaining: 1m 51s
195:	learn: 0.3964780	test: 0.3968365	best: 0.3968365 (195)	total: 27.2s	remaining: 1m 51s
196:	learn: 0.3964657	test: 0.3968327	best: 0.3968327 (196)	total: 27.3s	remaining: 1m 51s
1

277:	learn: 0.3958735	test: 0.3966779	best: 0.3966736 (274)	total: 38.7s	remaining: 1m 40s
278:	learn: 0.3958647	test: 0.3966758	best: 0.3966736 (274)	total: 38.9s	remaining: 1m 40s
279:	learn: 0.3958536	test: 0.3966724	best: 0.3966724 (279)	total: 39s	remaining: 1m 40s
280:	learn: 0.3958527	test: 0.3966710	best: 0.3966710 (280)	total: 39.1s	remaining: 1m 40s
281:	learn: 0.3958496	test: 0.3966720	best: 0.3966710 (280)	total: 39.2s	remaining: 1m 39s
282:	learn: 0.3958469	test: 0.3966730	best: 0.3966710 (280)	total: 39.4s	remaining: 1m 39s
283:	learn: 0.3958352	test: 0.3966720	best: 0.3966710 (280)	total: 39.5s	remaining: 1m 39s
284:	learn: 0.3958289	test: 0.3966688	best: 0.3966688 (284)	total: 39.7s	remaining: 1m 39s
285:	learn: 0.3958223	test: 0.3966674	best: 0.3966674 (285)	total: 39.8s	remaining: 1m 39s
286:	learn: 0.3958143	test: 0.3966626	best: 0.3966626 (286)	total: 39.9s	remaining: 1m 39s
287:	learn: 0.3958097	test: 0.3966646	best: 0.3966626 (286)	total: 40s	remaining: 1m 38s
288

369:	learn: 0.3951984	test: 0.3964954	best: 0.3964954 (369)	total: 51.1s	remaining: 1m 27s
370:	learn: 0.3951889	test: 0.3964936	best: 0.3964936 (370)	total: 51.3s	remaining: 1m 26s
371:	learn: 0.3951827	test: 0.3964907	best: 0.3964907 (371)	total: 51.4s	remaining: 1m 26s
372:	learn: 0.3951742	test: 0.3964910	best: 0.3964907 (371)	total: 51.5s	remaining: 1m 26s
373:	learn: 0.3951710	test: 0.3964906	best: 0.3964906 (373)	total: 51.7s	remaining: 1m 26s
374:	learn: 0.3951589	test: 0.3964841	best: 0.3964841 (374)	total: 51.8s	remaining: 1m 26s
375:	learn: 0.3951534	test: 0.3964850	best: 0.3964841 (374)	total: 51.9s	remaining: 1m 26s
376:	learn: 0.3951422	test: 0.3964805	best: 0.3964805 (376)	total: 52.1s	remaining: 1m 26s
377:	learn: 0.3951334	test: 0.3964805	best: 0.3964805 (377)	total: 52.2s	remaining: 1m 25s
378:	learn: 0.3951183	test: 0.3964676	best: 0.3964676 (378)	total: 52.4s	remaining: 1m 25s
379:	learn: 0.3951102	test: 0.3964623	best: 0.3964623 (379)	total: 52.5s	remaining: 1m 25s

...

553:	learn: 0.3940180	test: 0.3963802	best: 0.3963603 (486)	total: 1m 15s	remaining: 1m
554:	learn: 0.3940109	test: 0.3963785	best: 0.3963603 (486)	total: 1m 15s	remaining: 1m
555:	learn: 0.3940026	test: 0.3963785	best: 0.3963603 (486)	total: 1m 15s	remaining: 1m
556:	learn: 0.3939985	test: 0.3963781	best: 0.3963603 (486)	total: 1m 16s	remaining: 1m
557:	learn: 0.3939913	test: 0.3963796	best: 0.3963603 (486)	total: 1m 16s	remaining: 1m
558:	learn: 0.3939871	test: 0.3963794	best: 0.3963603 (486)	total: 1m 16s	remaining: 1m
559:	learn: 0.3939825	test: 0.3963825	best: 0.3963603 (486)	total: 1m 16s	remaining: 1m
560:	learn: 0.3939805	test: 0.3963786	best: 0.3963603 (486)	total: 1m 16s	remaining: 59.9s
561:	learn: 0.3939709	test: 0.3963782	best: 0.3963603 (486)	total: 1m 16s	remaining: 59.8s
562:	learn: 0.3939612	test: 0.3963789	best: 0.3963603 (486)	total: 1m 16s	remaining: 59.6s
563:	learn: 0.3939538	test: 0.3963768	best: 0.3963603 (486)	total: 1m 16s	remaining: 59.5s
564:	learn: 0.393950

645:	learn: 0.3935509	test: 0.3963384	best: 0.3963327 (643)	total: 1m 27s	remaining: 48.2s
646:	learn: 0.3935466	test: 0.3963385	best: 0.3963327 (643)	total: 1m 28s	remaining: 48.1s
647:	learn: 0.3935407	test: 0.3963363	best: 0.3963327 (643)	total: 1m 28s	remaining: 47.9s
648:	learn: 0.3935341	test: 0.3963358	best: 0.3963327 (643)	total: 1m 28s	remaining: 47.8s
649:	learn: 0.3935325	test: 0.3963363	best: 0.3963327 (643)	total: 1m 28s	remaining: 47.7s
650:	learn: 0.3935272	test: 0.3963348	best: 0.3963327 (643)	total: 1m 28s	remaining: 47.5s
651:	learn: 0.3935248	test: 0.3963354	best: 0.3963327 (643)	total: 1m 28s	remaining: 47.4s
652:	learn: 0.3935192	test: 0.3963343	best: 0.3963327 (643)	total: 1m 28s	remaining: 47.2s
653:	learn: 0.3935097	test: 0.3963309	best: 0.3963309 (653)	total: 1m 29s	remaining: 47.1s
654:	learn: 0.3935047	test: 0.3963328	best: 0.3963309 (653)	total: 1m 29s	remaining: 47s
655:	learn: 0.3934988	test: 0.3963330	best: 0.3963309 (653)	total: 1m 29s	remaining: 46.8s
6

737:	learn: 0.3930863	test: 0.3963121	best: 0.3963121 (737)	total: 1m 40s	remaining: 35.6s
738:	learn: 0.3930842	test: 0.3963129	best: 0.3963121 (737)	total: 1m 40s	remaining: 35.5s
739:	learn: 0.3930686	test: 0.3963107	best: 0.3963107 (739)	total: 1m 40s	remaining: 35.3s
740:	learn: 0.3930648	test: 0.3963114	best: 0.3963107 (739)	total: 1m 40s	remaining: 35.2s
741:	learn: 0.3930585	test: 0.3963087	best: 0.3963087 (741)	total: 1m 40s	remaining: 35.1s
742:	learn: 0.3930490	test: 0.3963051	best: 0.3963051 (742)	total: 1m 41s	remaining: 34.9s
743:	learn: 0.3930424	test: 0.3963070	best: 0.3963051 (742)	total: 1m 41s	remaining: 34.8s
744:	learn: 0.3930373	test: 0.3963190	best: 0.3963051 (742)	total: 1m 41s	remaining: 34.7s
745:	learn: 0.3930316	test: 0.3963207	best: 0.3963051 (742)	total: 1m 41s	remaining: 34.5s
746:	learn: 0.3930273	test: 0.3963193	best: 0.3963051 (742)	total: 1m 41s	remaining: 34.4s
747:	learn: 0.3930186	test: 0.3963202	best: 0.3963051 (742)	total: 1m 41s	remaining: 34.3s

829:	learn: 0.3925761	test: 0.3962866	best: 0.3962763 (807)	total: 1m 52s	remaining: 23.1s
830:	learn: 0.3925752	test: 0.3962863	best: 0.3962763 (807)	total: 1m 52s	remaining: 23s
831:	learn: 0.3925703	test: 0.3962855	best: 0.3962763 (807)	total: 1m 53s	remaining: 22.8s
832:	learn: 0.3925630	test: 0.3962808	best: 0.3962763 (807)	total: 1m 53s	remaining: 22.7s
833:	learn: 0.3925618	test: 0.3962803	best: 0.3962763 (807)	total: 1m 53s	remaining: 22.6s
834:	learn: 0.3925592	test: 0.3962809	best: 0.3962763 (807)	total: 1m 53s	remaining: 22.4s
835:	learn: 0.3925564	test: 0.3962784	best: 0.3962763 (807)	total: 1m 53s	remaining: 22.3s
836:	learn: 0.3925510	test: 0.3962793	best: 0.3962763 (807)	total: 1m 53s	remaining: 22.2s
837:	learn: 0.3925465	test: 0.3962777	best: 0.3962763 (807)	total: 1m 53s	remaining: 22s
838:	learn: 0.3925410	test: 0.3962771	best: 0.3962763 (807)	total: 1m 54s	remaining: 21.9s
839:	learn: 0.3925402	test: 0.3962773	best: 0.3962763 (807)	total: 1m 54s	remaining: 21.8s
840

921:	learn: 0.3921046	test: 0.3962202	best: 0.3962118 (913)	total: 2m 4s	remaining: 10.6s
922:	learn: 0.3921019	test: 0.3962187	best: 0.3962118 (913)	total: 2m 5s	remaining: 10.4s
923:	learn: 0.3920935	test: 0.3962220	best: 0.3962118 (913)	total: 2m 5s	remaining: 10.3s
924:	learn: 0.3920887	test: 0.3962228	best: 0.3962118 (913)	total: 2m 5s	remaining: 10.2s
925:	learn: 0.3920836	test: 0.3962231	best: 0.3962118 (913)	total: 2m 5s	remaining: 10s
926:	learn: 0.3920769	test: 0.3962231	best: 0.3962118 (913)	total: 2m 5s	remaining: 9.89s
927:	learn: 0.3920664	test: 0.3962286	best: 0.3962118 (913)	total: 2m 5s	remaining: 9.75s
928:	learn: 0.3920654	test: 0.3962293	best: 0.3962118 (913)	total: 2m 5s	remaining: 9.62s
929:	learn: 0.3920616	test: 0.3962273	best: 0.3962118 (913)	total: 2m 5s	remaining: 9.48s
930:	learn: 0.3920534	test: 0.3962271	best: 0.3962118 (913)	total: 2m 6s	remaining: 9.34s
931:	learn: 0.3920478	test: 0.3962271	best: 0.3962118 (913)	total: 2m 6s	remaining: 9.21s
932:	learn: 

In [23]:
clf = CatBoostClassifier(task_type ='GPU', loss_function='Logloss', eval_metric = 'Logloss', scale_pos_weight=4.874)
clf.fit(X_train, y_train, cat_features = categorical_feature_indices, eval_set = (X_val, y_val))
print(clf.get_best_score())

Learning rate set to 0.157426
0:	learn: 0.6652968	test: 0.6653288	best: 0.6653288 (0)	total: 120ms	remaining: 2m
1:	learn: 0.6456065	test: 0.6451919	best: 0.6451919 (1)	total: 268ms	remaining: 2m 13s
2:	learn: 0.6318544	test: 0.6313792	best: 0.6313792 (2)	total: 404ms	remaining: 2m 14s
3:	learn: 0.6224232	test: 0.6217981	best: 0.6217981 (3)	total: 538ms	remaining: 2m 13s
4:	learn: 0.6156136	test: 0.6150266	best: 0.6150266 (4)	total: 664ms	remaining: 2m 12s
5:	learn: 0.6108719	test: 0.6102031	best: 0.6102031 (5)	total: 804ms	remaining: 2m 13s
6:	learn: 0.6069471	test: 0.6063338	best: 0.6063338 (6)	total: 932ms	remaining: 2m 12s
7:	learn: 0.6042462	test: 0.6036587	best: 0.6036587 (7)	total: 1.05s	remaining: 2m 10s
8:	learn: 0.6021565	test: 0.6016021	best: 0.6016021 (8)	total: 1.2s	remaining: 2m 11s
9:	learn: 0.6004822	test: 0.5999629	best: 0.5999629 (9)	total: 1.33s	remaining: 2m 11s
10:	learn: 0.5991021	test: 0.5986056	best: 0.5986056 (10)	total: 1.46s	remaining: 2m 11s
11:	learn: 0.598

94:	learn: 0.5876611	test: 0.5890611	best: 0.5890611 (94)	total: 12.3s	remaining: 1m 57s
95:	learn: 0.5876254	test: 0.5890587	best: 0.5890587 (95)	total: 12.5s	remaining: 1m 57s
96:	learn: 0.5875911	test: 0.5890418	best: 0.5890418 (96)	total: 12.6s	remaining: 1m 57s
97:	learn: 0.5875420	test: 0.5890266	best: 0.5890266 (97)	total: 12.8s	remaining: 1m 57s
98:	learn: 0.5874893	test: 0.5890208	best: 0.5890208 (98)	total: 12.9s	remaining: 1m 57s
99:	learn: 0.5874415	test: 0.5889680	best: 0.5889680 (99)	total: 13s	remaining: 1m 57s
100:	learn: 0.5873535	test: 0.5889152	best: 0.5889152 (100)	total: 13.1s	remaining: 1m 56s
101:	learn: 0.5873246	test: 0.5889054	best: 0.5889054 (101)	total: 13.2s	remaining: 1m 56s
102:	learn: 0.5872821	test: 0.5888577	best: 0.5888577 (102)	total: 13.4s	remaining: 1m 56s
103:	learn: 0.5872518	test: 0.5888366	best: 0.5888366 (103)	total: 13.5s	remaining: 1m 56s
104:	learn: 0.5872332	test: 0.5888317	best: 0.5888317 (104)	total: 13.7s	remaining: 1m 56s
105:	learn: 0

186:	learn: 0.5849669	test: 0.5880349	best: 0.5880349 (186)	total: 24.6s	remaining: 1m 46s
187:	learn: 0.5849295	test: 0.5880090	best: 0.5880090 (187)	total: 24.7s	remaining: 1m 46s
188:	learn: 0.5849102	test: 0.5879926	best: 0.5879926 (188)	total: 24.8s	remaining: 1m 46s
189:	learn: 0.5848964	test: 0.5879904	best: 0.5879904 (189)	total: 24.9s	remaining: 1m 46s
190:	learn: 0.5848754	test: 0.5879882	best: 0.5879882 (190)	total: 25.1s	remaining: 1m 46s
191:	learn: 0.5848531	test: 0.5879827	best: 0.5879827 (191)	total: 25.2s	remaining: 1m 46s
192:	learn: 0.5848375	test: 0.5879770	best: 0.5879770 (192)	total: 25.3s	remaining: 1m 45s
193:	learn: 0.5848194	test: 0.5879762	best: 0.5879762 (193)	total: 25.5s	remaining: 1m 45s
194:	learn: 0.5848060	test: 0.5879890	best: 0.5879762 (193)	total: 25.6s	remaining: 1m 45s
195:	learn: 0.5847893	test: 0.5879828	best: 0.5879762 (193)	total: 25.7s	remaining: 1m 45s
196:	learn: 0.5847673	test: 0.5879746	best: 0.5879746 (196)	total: 25.8s	remaining: 1m 45s

278:	learn: 0.5830919	test: 0.5876157	best: 0.5876079 (273)	total: 36.7s	remaining: 1m 34s
279:	learn: 0.5830804	test: 0.5876068	best: 0.5876068 (279)	total: 36.9s	remaining: 1m 34s
280:	learn: 0.5830601	test: 0.5876066	best: 0.5876066 (280)	total: 37s	remaining: 1m 34s
281:	learn: 0.5830361	test: 0.5876045	best: 0.5876045 (281)	total: 37.1s	remaining: 1m 34s
282:	learn: 0.5830151	test: 0.5876017	best: 0.5876017 (282)	total: 37.3s	remaining: 1m 34s
283:	learn: 0.5830026	test: 0.5876012	best: 0.5876012 (283)	total: 37.4s	remaining: 1m 34s
284:	learn: 0.5829879	test: 0.5876050	best: 0.5876012 (283)	total: 37.5s	remaining: 1m 34s
285:	learn: 0.5829731	test: 0.5875944	best: 0.5875944 (285)	total: 37.7s	remaining: 1m 34s
286:	learn: 0.5829565	test: 0.5875830	best: 0.5875830 (286)	total: 37.8s	remaining: 1m 33s
287:	learn: 0.5829361	test: 0.5875739	best: 0.5875739 (287)	total: 37.9s	remaining: 1m 33s
288:	learn: 0.5829268	test: 0.5875669	best: 0.5875669 (288)	total: 38.1s	remaining: 1m 33s
2

370:	learn: 0.5816542	test: 0.5874285	best: 0.5874285 (370)	total: 49s	remaining: 1m 23s
371:	learn: 0.5816398	test: 0.5874267	best: 0.5874267 (371)	total: 49.2s	remaining: 1m 22s
372:	learn: 0.5816330	test: 0.5874201	best: 0.5874201 (372)	total: 49.3s	remaining: 1m 22s
373:	learn: 0.5816115	test: 0.5874158	best: 0.5874158 (373)	total: 49.5s	remaining: 1m 22s
374:	learn: 0.5815934	test: 0.5874240	best: 0.5874158 (373)	total: 49.6s	remaining: 1m 22s
375:	learn: 0.5815739	test: 0.5874202	best: 0.5874158 (373)	total: 49.7s	remaining: 1m 22s
376:	learn: 0.5815590	test: 0.5874130	best: 0.5874130 (376)	total: 49.9s	remaining: 1m 22s
377:	learn: 0.5815304	test: 0.5874131	best: 0.5874130 (376)	total: 50s	remaining: 1m 22s
378:	learn: 0.5815192	test: 0.5874103	best: 0.5874103 (378)	total: 50.2s	remaining: 1m 22s
379:	learn: 0.5815078	test: 0.5874700	best: 0.5874103 (378)	total: 50.3s	remaining: 1m 22s
380:	learn: 0.5814834	test: 0.5874666	best: 0.5874103 (378)	total: 50.4s	remaining: 1m 21s
381

...

554:	learn: 0.5790213	test: 0.5873422	best: 0.5873422 (554)	total: 1m 13s	remaining: 58.8s
555:	learn: 0.5790069	test: 0.5873413	best: 0.5873413 (555)	total: 1m 13s	remaining: 58.7s
556:	learn: 0.5789892	test: 0.5873410	best: 0.5873410 (556)	total: 1m 13s	remaining: 58.5s
557:	learn: 0.5789788	test: 0.5873385	best: 0.5873385 (557)	total: 1m 13s	remaining: 58.4s
558:	learn: 0.5789686	test: 0.5873338	best: 0.5873338 (558)	total: 1m 13s	remaining: 58.3s
559:	learn: 0.5789608	test: 0.5873291	best: 0.5873291 (559)	total: 1m 14s	remaining: 58.2s
560:	learn: 0.5789466	test: 0.5873341	best: 0.5873291 (559)	total: 1m 14s	remaining: 58s
561:	learn: 0.5789310	test: 0.5873325	best: 0.5873291 (559)	total: 1m 14s	remaining: 57.9s
562:	learn: 0.5789175	test: 0.5873235	best: 0.5873235 (562)	total: 1m 14s	remaining: 57.8s
563:	learn: 0.5789092	test: 0.5873192	best: 0.5873192 (563)	total: 1m 14s	remaining: 57.6s
564:	learn: 0.5789023	test: 0.5873201	best: 0.5873192 (563)	total: 1m 14s	remaining: 57.5s
5

646:	learn: 0.5777790	test: 0.5873124	best: 0.5872837 (581)	total: 1m 25s	remaining: 46.8s
647:	learn: 0.5777559	test: 0.5873048	best: 0.5872837 (581)	total: 1m 26s	remaining: 46.7s
648:	learn: 0.5777352	test: 0.5873024	best: 0.5872837 (581)	total: 1m 26s	remaining: 46.6s
649:	learn: 0.5777224	test: 0.5872974	best: 0.5872837 (581)	total: 1m 26s	remaining: 46.5s
650:	learn: 0.5777064	test: 0.5872962	best: 0.5872837 (581)	total: 1m 26s	remaining: 46.3s
651:	learn: 0.5777005	test: 0.5872989	best: 0.5872837 (581)	total: 1m 26s	remaining: 46.2s
652:	learn: 0.5776817	test: 0.5872914	best: 0.5872837 (581)	total: 1m 26s	remaining: 46.1s
653:	learn: 0.5776651	test: 0.5872839	best: 0.5872837 (581)	total: 1m 26s	remaining: 46s
654:	learn: 0.5776573	test: 0.5872957	best: 0.5872837 (581)	total: 1m 27s	remaining: 45.8s
655:	learn: 0.5776433	test: 0.5872966	best: 0.5872837 (581)	total: 1m 27s	remaining: 45.7s
656:	learn: 0.5776346	test: 0.5872945	best: 0.5872837 (581)	total: 1m 27s	remaining: 45.6s
6

738:	learn: 0.5765265	test: 0.5873006	best: 0.5872659 (697)	total: 1m 38s	remaining: 34.7s
739:	learn: 0.5765200	test: 0.5873000	best: 0.5872659 (697)	total: 1m 38s	remaining: 34.6s
740:	learn: 0.5765001	test: 0.5872962	best: 0.5872659 (697)	total: 1m 38s	remaining: 34.5s
741:	learn: 0.5764793	test: 0.5872972	best: 0.5872659 (697)	total: 1m 38s	remaining: 34.3s
742:	learn: 0.5764613	test: 0.5872900	best: 0.5872659 (697)	total: 1m 38s	remaining: 34.2s
743:	learn: 0.5764505	test: 0.5872862	best: 0.5872659 (697)	total: 1m 38s	remaining: 34.1s
744:	learn: 0.5764456	test: 0.5872859	best: 0.5872659 (697)	total: 1m 39s	remaining: 33.9s
745:	learn: 0.5764303	test: 0.5872736	best: 0.5872659 (697)	total: 1m 39s	remaining: 33.8s
746:	learn: 0.5764236	test: 0.5872719	best: 0.5872659 (697)	total: 1m 39s	remaining: 33.7s
747:	learn: 0.5764076	test: 0.5872673	best: 0.5872659 (697)	total: 1m 39s	remaining: 33.5s
748:	learn: 0.5763940	test: 0.5872790	best: 0.5872659 (697)	total: 1m 39s	remaining: 33.4s

830:	learn: 0.5753909	test: 0.5873169	best: 0.5872659 (697)	total: 1m 50s	remaining: 22.5s
831:	learn: 0.5753798	test: 0.5873152	best: 0.5872659 (697)	total: 1m 50s	remaining: 22.4s
832:	learn: 0.5753637	test: 0.5873158	best: 0.5872659 (697)	total: 1m 50s	remaining: 22.2s
833:	learn: 0.5753408	test: 0.5873219	best: 0.5872659 (697)	total: 1m 51s	remaining: 22.1s
834:	learn: 0.5753243	test: 0.5873225	best: 0.5872659 (697)	total: 1m 51s	remaining: 22s
835:	learn: 0.5753151	test: 0.5873253	best: 0.5872659 (697)	total: 1m 51s	remaining: 21.9s
836:	learn: 0.5753044	test: 0.5873312	best: 0.5872659 (697)	total: 1m 51s	remaining: 21.7s
837:	learn: 0.5752923	test: 0.5873315	best: 0.5872659 (697)	total: 1m 51s	remaining: 21.6s
838:	learn: 0.5752801	test: 0.5873344	best: 0.5872659 (697)	total: 1m 51s	remaining: 21.5s
839:	learn: 0.5752727	test: 0.5873381	best: 0.5872659 (697)	total: 1m 51s	remaining: 21.3s
840:	learn: 0.5752593	test: 0.5873383	best: 0.5872659 (697)	total: 1m 52s	remaining: 21.2s
8

922:	learn: 0.5743176	test: 0.5873911	best: 0.5872659 (697)	total: 2m 3s	remaining: 10.3s
923:	learn: 0.5743122	test: 0.5873897	best: 0.5872659 (697)	total: 2m 3s	remaining: 10.1s
924:	learn: 0.5743042	test: 0.5873898	best: 0.5872659 (697)	total: 2m 3s	remaining: 10s
925:	learn: 0.5742983	test: 0.5873906	best: 0.5872659 (697)	total: 2m 3s	remaining: 9.87s
926:	learn: 0.5742799	test: 0.5873946	best: 0.5872659 (697)	total: 2m 3s	remaining: 9.73s
927:	learn: 0.5742679	test: 0.5873948	best: 0.5872659 (697)	total: 2m 3s	remaining: 9.6s
928:	learn: 0.5742574	test: 0.5873958	best: 0.5872659 (697)	total: 2m 3s	remaining: 9.47s
929:	learn: 0.5742474	test: 0.5873899	best: 0.5872659 (697)	total: 2m 4s	remaining: 9.33s
930:	learn: 0.5742325	test: 0.5873929	best: 0.5872659 (697)	total: 2m 4s	remaining: 9.2s
931:	learn: 0.5742159	test: 0.5874024	best: 0.5872659 (697)	total: 2m 4s	remaining: 9.07s
932:	learn: 0.5741946	test: 0.5873862	best: 0.5872659 (697)	total: 2m 4s	remaining: 8.94s
933:	learn: 0.

In [20]:
categorical_feature_indices

array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
      dtype=int64)